# Aqueduct Food Widgets

In [1]:
import pandas as pd
import json
import requests
import numpy as np
import ipywidgets as widgets

In [ ]:
     b as (SELECT initcap(split_part(risk, ' (', 1)) as risk, replace(initcap(indicator), '_', ' ') as indicator, area_perc, perc_value, irrigation
           FROM a
           WHERE (indicator='water_stress' and irrigation='irrigated') or (indicator='seasonal_variability' and irrigation='rainfed') or 
                 (indicator='interannual_variability' and irrigation='irrigated') or 
                 (indicator='coastal_eutrophication_potential' and irrigation='irrigated')),


b as (SELECT initcap(split_part(risk, ' (', 1)) as risk, replace(initcap(indicator), '_', ' ') as indicator, area_perc, perc_value, irrigation FROM a WHERE (indicator='water_stress' and irrigation='irrigated') or (indicator='seasonal_variability' and irrigation='rainfed') or (indicator='interannual_variability' and irrigation='irrigated') or (indicator='coastal_eutrophication_potential' and irrigation='irrigated')),


# Widgets' structure
## All crops (staging)
### GLOBAL

- <a href="#World-Price">World Price</a>
    * **Widget_id:** [9391e42c-f32b-40d9-a282-3083e617464f](https://staging-api.globalforestwatch.org/v1/widget/9391e42c-f32b-40d9-a282-3083e617464f)
- <a href="#Average-global-change-in-seasonal-variability-for-rainfed-crops-through-2040">Average global change in seasonal variability for rainfed crops through 2040</a>
    * **Widget_id:** [abf5c0f9-4042-4ea6-9582-1b34209cb4ba](https://staging-api.globalforestwatch.org/v1/widget/abf5c0f9-4042-4ea6-9582-1b34209cb4ba)
- <a href="#Percent-of-crop-area-by-level-of-water-stress-and-volume-of-demand">Percent of crop area by level of water stress and volume of demand</a>
    * **Widget_id:** [7b29a2a5-515a-4e22-846f-03ad3d2c0a13](https://staging-api.globalforestwatch.org/v1/widget/7b29a2a5-515a-4e22-846f-03ad3d2c0a13)
- <a href="#Average-global-change-in-water-stress-for-irrigated-crops-through-2040">Average global change in water stress for irrigated crops through 2040</a>
    * **Widget_id:** [4f322f4a-dde6-4f63-b7c9-10b0ba0c1402](https://staging-api.globalforestwatch.org/v1/widget/4f322f4a-dde6-4f63-b7c9-10b0ba0c1402)
- <a href="#Irrigated-vs-Rainfed">Irrigated vs Rainfed</a>
    * **Widget_id:** [606c813b-d716-4b0b-8717-9389425c9262](https://staging-api.globalforestwatch.org/v1/widget/606c813b-d716-4b0b-8717-9389425c9262)
- <a href="#Percentage-of-crop-production-area-by-water-risk-in-year">Percentage of crop production area by water risk in year</a>
    * **Widget_id:** [7b37bd6e-a465-4c4d-98b3-1a110a66d27e](https://staging-api.globalforestwatch.org/v1/widget/7b37bd6e-a465-4c4d-98b3-1a110a66d27e)
    
### COUNTRY

- <a href="#Percent-of-crop-area-by-level-of-water-stress-and-volume-of-demand-in-country">Percent of crop area by level of water stress and volume of demand in country</a> 
    * **Widget_id:** [e2778762-4b3d-498c-8b4a-869ac834f406](https://staging-api.globalforestwatch.org/v1/widget/e2778762-4b3d-498c-8b4a-869ac834f406) 
- Percentage of country's population at high risk of hunger
    * **Widget_id:** [8f67fadd-d8df-4a28-82dd-a22a337d71b9](https://staging-api.globalforestwatch.org/v1/widget/8f67fadd-d8df-4a28-82dd-a22a337d71b9)
- Kilocalories per capita
    * **Widget_id:** [af969fd8-fd47-430d-ad13-9113e42e9fed](https://staging-api.globalforestwatch.org/v1/widget/af969fd8-fd47-430d-ad13-9113e42e9fed) 
- Volume of production, food demand, and net trade
    * **Widget_id:** [eb14a35e-50bc-44c0-88fd-199324550468](https://staging-api.globalforestwatch.org/v1/widget/eb14a35e-50bc-44c0-88fd-199324550468) 
- Percentage of {{crop_name}} area that {{year_verb}}
    * **Widget_id:** [25facdf3-acf6-4f0e-9a0a-5ec94216407f](https://staging-api.globalforestwatch.org/v1/widget/25facdf3-acf6-4f0e-9a0a-5ec94216407f) 
- Change in net trade over time for top 5 crops in highest demand in {{countryName}}
    * **Widget_id:** [88aefca0-e25c-4a16-80b3-f2831d2064e4](https://staging-api.globalforestwatch.org/v1/widget/88aefca0-e25c-4a16-80b3-f2831d2064e4) 
- <a href="#Percentage-of-crop-production-area-by-water-risk-in-year-in-country">Percentage of crop production area by water risk in year in country</a>
    * **Widget_id:** [962fad79-ec09-4258-b233-d4934bfbbf61](https://staging-api.globalforestwatch.org/v1/widget/962fad79-ec09-4258-b233-d4934bfbbf61) 

## One crop (staging)
### GLOBAL

- <a href="#Single-growing-crop-under-high-water-stress-conditions">Single growing crop under high water stress conditions</a>
    * **Widget_id:** [641970f9-84da-4560-a715-ad4f70a3b8f4](https://staging-api.globalforestwatch.org/v1/widget/641970f9-84da-4560-a715-ad4f70a3b8f4)
- Irrigated vs Rainfed
    * **Widget_id:** [606c813b-d716-4b0b-8717-9389425c9262](https://staging-api.globalforestwatch.org/v1/widget/606c813b-d716-4b0b-8717-9389425c9262) 
- Top 5 net importers and net exporters of {{commodity}}
    * **Widget_id:** [4a3582e3-65d8-4331-b397-f42ed5d190d0](https://staging-api.globalforestwatch.org/v1/widget/4a3582e3-65d8-4331-b397-f42ed5d190d0)
- Top 5 countries with the greatest projected irrigated yield increases through {{year}}
    * **Widget_id:** [38b2cde3-aab9-4310-834f-9ac2a96a91a7](https://staging-api.globalforestwatch.org/v1/widget/38b2cde3-aab9-4310-834f-9ac2a96a91a7)
- <a href="#Top-10-countries-with-the-highest-crop-water-stress-scores-in-year">Top 10 countries with the highest crop water stress scores in year</a>
    * **Widget_id:** [413a36b5-a7b4-4bb4-9491-98b5175b4e28](https://staging-api.globalforestwatch.org/v1/widget/413a36b5-a7b4-4bb4-9491-98b5175b4e28)
- <a href="#Top-10-countries-with-the-highest-crop-drought-severity-risk-scores">Top 10 countries with the highest crop drought severity risk scores</a>
    * **Widget_id:** [43d3bfe3-802d-444b-a749-714e2569e2ae](https://staging-api.globalforestwatch.org/v1/widget/43d3bfe3-802d-444b-a749-714e2569e2ae)
- Global {{commodity}} yield, area, production, price, and demand change over time
    * **Widget_id:** [e0bdc2fc-e1a9-431e-ad91-5f2c9023f03f](https://staging-api.globalforestwatch.org/v1/widget/e0bdc2fc-e1a9-431e-ad91-5f2c9023f03f)
- Groundwater tables declined percentage in areas where a crop is grown since 1990
    * **Widget_id:** [b65cb119-051f-478b-9599-4f16df0c7c41](https://staging-api.globalforestwatch.org/v1/widget/b65cb119-051f-478b-9599-4f16df0c7c41)
- <a href="#On-average-risk-of-water-stress-will-increase-in-areas-where-crop-is-grown-by-x-through-year">On average risk of water stress will increase in areas where crop is grown by x through year</a>
    * **Widget_id:** [d171427d-ad95-4140-a9a4-7c24ea1019da](https://staging-api.globalforestwatch.org/v1/widget/d171427d-ad95-4140-a9a4-7c24ea1019da)
- <a href="#On-average-risk-of-seasonal-variability-will-increase-in-areas-where-crop-is-grown-by-x-through-year">On average risk of seasonal variability will increase in areas where crop is grown by x through year</a>
    * **Widget_id:** [b98c1774-9ce8-4ae5-b3e3-6431cc94921f](https://staging-api.globalforestwatch.org/v1/widget/b98c1774-9ce8-4ae5-b3e3-6431cc94921f)
- <a href="#Percentage-of-global-production-area-of-crop-by-water-risk-in-year">Percentage of global production area of crop by water risk in year</a>
    * **Widget_id:** [c1d1143f-af7d-4e18-990b-b8553423e5df](https://staging-api.globalforestwatch.org/v1/widget/c1d1143f-af7d-4e18-990b-b8553423e5df)
    
### COUNTRY

- <a href="#Country-summary-crop">Country summary crop</a>
    * **[Widget](https://github.com/Vizzuality/Aqueduct-Water_Food_Security/blob/feature%2Fnew-map/src/components/summary/constants.js)**
- Percentage of country's population at high risk of hunger.
    * **Widget_id:** [8f67fadd-d8df-4a28-82dd-a22a337d71b9](https://staging-api.globalforestwatch.org/v1/widget/8f67fadd-d8df-4a28-82dd-a22a337d71b9) 
- Volume of production, food demand, and net trade
    * **Widget_id:** [eb14a35e-50bc-44c0-88fd-199324550468](https://staging-api.globalforestwatch.org/v1/widget/eb14a35e-50bc-44c0-88fd-199324550468) 
- Percentage of {{crop_name}} area that {{year_verb}}
    * **Widget_id:** [25facdf3-acf6-4f0e-9a0a-5ec94216407f](https://staging-api.globalforestwatch.org/v1/widget/25facdf3-acf6-4f0e-9a0a-5ec94216407f) 
-  <a href="#Change-over-time-in-irrigated-crop-area-exposed-to-water-stress-and-rainfed-crop-area-exposed-to-seasonal-variability-in-country">Change over time in irrigated crop area exposed to water stress and rainfed crop area exposed to seasonal variability in country</a>
    * **Widget_id:** [69963620-4cc4-4010-9072-0315cbc71265](https://staging-api.globalforestwatch.org/v1/widget/69963620-4cc4-4010-9072-0315cbc71265)
- <a href="#Growing-crops-under-water-stress-conditions">Growing crops under water stress conditions</a>
    * **Widget_id:** [049f074a-3528-427d-922b-3c2320e9caf6](https://staging-api.globalforestwatch.org/v1/widget/049f074a-3528-427d-922b-3c2320e9caf6) 
- On average, groundwater tables have declined by x% in areas where [crop] is grown since 1990.
    * **Widget_id:** [ade06195-c4ce-4285-8d11-98f5688d1b22](https://staging-api.globalforestwatch.org/v1/widget/ade06195-c4ce-4285-8d11-98f5688d1b22) 
- <a href="#On-average-risk-of-water-stress-will-increase-in-areas-where-a-crop-is-grown-through-year">On average risk of water stress will increase in areas where a crop is grown through year</a>
    * **Widget_id:** [e5aef90b-79c5-4e61-a36c-7b8545864c4a](https://staging-api.globalforestwatch.org/v1/widget/e5aef90b-79c5-4e61-a36c-7b8545864c4a) 
- <a href="#On-average-risk-of-seasonal-variability-will-increase-in-areas-where-a-crop-is-grown-through-year">On average risk of seasonal variability will increase in areas where a crop is grown through year</a>
    * **Widget_id:** [5e0f0039-48c2-4d17-93b6-cbb2535c22b5](https://staging-api.globalforestwatch.org/v1/widget/5e0f0039-48c2-4d17-93b6-cbb2535c22b5) 
- {{crop_name}} yield, area, production, and demand change over time in {{countryName}}
    * **Widget_id:** [d654b935-31cd-4971-877e-bf0345c8b49a](https://staging-api.globalforestwatch.org/v1/widget/d654b935-31cd-4971-877e-bf0345c8b49a) 
- Change in {{countryName}}’s net trade of {{commodity}} over time
    * **Widget_id:** [636d41ed-620b-4da9-8796-e5783347fc10](https://staging-api.globalforestwatch.org/v1/widget/636d41ed-620b-4da9-8796-e5783347fc10)
- Percentage of global {{crop_name}} production area by water risk in {{countryName}} in {{year}}
    * **Widget_id:** [12362ca1-6f1a-43d3-8aaf-b25f1a22ed00](https://staging-api.globalforestwatch.org/v1/widget/12362ca1-6f1a-43d3-8aaf-b25f1a22ed00)

# Widget Config
## All crops (staging)
### GLOBAL
#### World Price
**Widget_id:** [9391e42c-f32b-40d9-a282-3083e617464f](https://staging-api.globalforestwatch.org/v1/widget/9391e42c-f32b-40d9-a282-3083e617464f)
    
**Old query:**
    
`url: https://wri-01.carto.com/api/v2/sql?q=`

`Sql:`

```sql
SELECT round((sum(value))::numeric,2) as val,avg(value) as val_avg, max_value, commodity, impactparameter, region, '2005 usd/Ton' as units 
FROM (SELECT  max(value) over (partition by impactparameter) max_value, * 
      FROM combined01_prepared 
      WHERE scenario = 'SSP2-MIRO' and impactparameter = 'World price') r 
{{where}} and commodity IN  ('Maize','Rice','Soybean','Wheat') 
GROUP BY max_value, region, commodity, impactparameter 
ORDER BY impactparameter asc, commodity asc 
LIMIT 5
```

`sql_config:`
```json
"sql_config": [
    {
        "key": "where",
        "key_params": [
            {
                "key": "year",
                "required": true
            }
        ]
    }
]
```

In [4]:
payload = {'q': "select round((sum(value))::numeric,2) as val,avg(value) as val_avg, max_value, commodity, impactparameter, region, '2005 usd/Ton' as units from (SELECT  max(value) over (partition by impactparameter) max_value, * FROM combined01_prepared where scenario = 'SSP2-MIRO' and impactparameter = 'World price') r where year = 2010  and commodity IN  ('Maize','Rice','Soybean','Wheat') group by max_value, region, commodity, impactparameter order by impactparameter asc, commodity asc limit 5"}
r = requests.get('https://wri-01.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df

<Response [200]>


,commodity,impactparameter,max_value,region,units,val,val_avg
0,Maize,World price,2296.737549,-,2005 usd/Ton,148.66,148.657593
1,Rice,World price,2296.737549,-,2005 usd/Ton,357.08,357.076416
2,Soybean,World price,2296.737549,-,2005 usd/Ton,331.40,331.399841
3,Wheat,World price,2296.737549,-,2005 usd/Ton,219.44,219.442307


**New query:**

`url: https://wri-01.carto.com/api/v2/sql?q=`

`Sql:`

```sql
SELECT round((sum(value))::numeric,2) as val,avg(value) as val_avg, max_value, commodity, impactparameter, region, '2005 usd/Ton' as units 
FROM (SELECT  max(value) over (partition by impactparameter) max_value, * 
      FROM combined01_prepared_new 
      WHERE scenario = 'SSP2-MIRO' and impactparameter = 'World price') r 
{{where}} and commodity IN  ('maize','rice','soybean','wheat') 
GROUP BY max_value, region, commodity, impactparameter 
ORDER BY impactparameter asc, commodity asc 
LIMIT 5
```

`sql_config:`
```json
"sql_config": [
    {
        "key": "where",
        "key_params": [
            {
                "key": "year",
                "required": true
            }
        ]
    }
]
```

In [5]:
payload = {'q': "select round((sum(value))::numeric,2) as val,avg(value) as val_avg, max_value, commodity, impactparameter, region, '2005 usd/Ton' as units from (SELECT  max(value) over (partition by impactparameter) max_value, * FROM combined01_prepared_new where scenario = 'SSP2-MIRO' and impactparameter = 'World price') r where year = 2010  and commodity IN  ('maize','rice','soybean','wheat') group by max_value, region, commodity, impactparameter order by impactparameter asc, commodity asc limit 5"}
r = requests.get('https://wri-01.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df.head()

<Response [200]>


,commodity,impactparameter,max_value,region,units,val,val_avg
0,maize,World price,2296.737549,-,2005 usd/Ton,148.66,148.657593
1,rice,World price,2296.737549,-,2005 usd/Ton,357.08,357.076416
2,soybean,World price,2296.737549,-,2005 usd/Ton,331.40,331.399841
3,wheat,World price,2296.737549,-,2005 usd/Ton,219.44,219.442307


#### Average global change in seasonal variability for rainfed crops through 2040
**Widget_id:** [abf5c0f9-4042-4ea6-9582-1b34209cb4ba](https://staging-api.globalforestwatch.org/v1/widget/abf5c0f9-4042-4ea6-9582-1b34209cb4ba)
    
**Old query:**
    
`url: https://wri-rw.carto.com/api/v2/sql?q=`

`Sql:`
```sql
with f as (SELECT basinid, crop, irrigation, iso 
           FROM crops 
           WHERE irrigation='rainfed'), 
     g as (SELECT basinid, sv3028tr, sv2028tr, sv4028tr, svbs00tr 
           FROM aqueduct_projections_20150309  
           where  svbs00tr > -1), 
     a as (SELECT avg(sv3028tr) as sv2030, avg(sv2028tr) as sv2020, avg(sv4028tr) as sv2040, avg(svbs00tr) as sv2010, crop 
           FROM f,g 
           WHERE f.basinid=g.basinid 
           GROUP BY crop), 
     b as (SELECT sv2010, sv2020, sv2030, sv2040, crop, 
           CASE when crop IN ('barley','maize','millet','rice','sorghum','wheat','other cereals') then 'Cereal' 
                when crop IN ('beans', 'cowpeas', 'chickpeas', 'groundnut', 'lentils', 'pigeonpeas', 'other pulses') then 'Pulses and legumes' 
                when crop IN ('cassava','potato','sweet potato','yams') then 'Roots And Tubers' 
                when crop IN ('banana','plantain') then 'Fruit and nuts' 
                when crop IN ('soybean') then 'Oilseed crops' 
           END as type 
           FROM a t), 
     c as (SELECT avg(sv2010) as sv2010,avg(sv2020) as sv2020,avg(sv2030) as sv2030,avg(sv2040) as sv2040, type 
           FROM b 
           GROUP BY type), 
   rrr as (SELECT type as cat, unnest(array[ '2010', '2020', '2030', '2040']) AS year, 
                  unnest(array[ sv2010, sv2020, sv2030, sv2040]) AS water_risk 
           FROM c order by year, cat) 
                               
                               
   SELECT year, cat as impactparameter, water_risk as value 
   FROM rrr 
   WHERE cat IN ('Cereal', 'Pulses and legumes', 'Roots And Tubers', 'Fruit and nuts', 'Oilseed crops') 
   ORDER BY year, 
            CASE cat WHEN 'Cereal' THEN 1 
                     WHEN 'Pulses and legumes' THEN 2 
                     WHEN 'Roots And Tubers' THEN 3 
                     WHEN 'Fruit and nuts' THEN 4 
            ELSE 5 
            END
```

In [6]:
payload = {'q': "with f as(SELECT basinid, crop, irrigation, iso FROM crops where irrigation='rainfed'), g as (SELECT basinid, sv3028tr, sv2028tr, sv4028tr, svbs00tr FROM aqueduct_projections_20150309  where  svbs00tr > -1), a as(select avg(sv3028tr) as sv2030, avg(sv2028tr) as sv2020, avg(sv4028tr) as sv2040, avg(svbs00tr) as sv2010, crop from f,g where f.basinid=g.basinid group by crop), b as (SELECT sv2010, sv2020, sv2030, sv2040, crop, CASE when crop IN ('barley','maize','millet','rice','sorghum','wheat','other cereals') then 'Cereal' when crop IN ('beans', 'cowpeas', 'chickpeas', 'groundnut', 'lentils', 'pigeonpeas', 'other pulses') then 'Pulses and legumes' when crop IN ('cassava','potato','sweet potato','yams') then 'Roots And Tubers' when crop IN ('banana','plantain') then 'Fruit and nuts' when crop IN ('soybean') then 'Oilseed crops' END as type FROM a t), c as (select avg(sv2010) as sv2010,avg(sv2020) as sv2020,avg(sv2030) as sv2030,avg(sv2040) as sv2040, type from b group by type), rrr as (SELECT type as cat, unnest(array[ '2010', '2020', '2030', '2040']) AS \"year\", unnest(array[ sv2010, sv2020, sv2030, sv2040]) AS \"water_risk\" FROM c order by year, cat) select year, cat as impactparameter, water_risk as value FROM rrr WHERE cat IN ('Cereal', 'Pulses and legumes', 'Roots And Tubers', 'Fruit and nuts', 'Oilseed crops') ORDER BY year, CASE cat WHEN 'Cereal' THEN 1 WHEN 'Pulses and legumes' THEN 2 WHEN 'Roots And Tubers' THEN 3 WHEN 'Fruit and nuts' THEN 4 ELSE 5 END"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df[df['year'] == 2010]

<Response [200]>


,impactparameter,value,year
0,Cereal,0.911941,2010
1,Pulses and legumes,1.084655,2010
2,Roots And Tubers,0.911385,2010
3,Fruit and nuts,0.902324,2010
4,Oilseed crops,0.797071,2010


**New query:**

First we create an intermediate table called `crops_risk_rankings`:
```sql
with bws as (SELECT r.crop, r.irrigation, r.area, r.prod, r.iso, r.name_cntr as region, 'water_stress' as indicator, 2014 as year,
                    coalesce(bws_score, -9999) as  value, coalesce(bws_cat, -9999) as  cat
             FROM water_risk_indicators_annual_all s
             LEFT JOIN crops_baseline r on s.aq30_id=r.aq30_id
             WHERE s.pfaf_id != -9999 and s.gid_1 != '-9999' and  bws_score != -9999 and  bws_cat != -9999),
bws_rank as (SELECT sum(value*area)/sum(area) as value, round(sum(cat*area)/sum(area)) as cat, crop, irrigation, iso, region, indicator, year
             FROM bws
             GROUP BY crop, irrigation, iso, region, indicator, year),
bws_label as(SELECT value, crop, irrigation, iso, region, indicator, year,
                    CASE when cat = 0 then 'low stress'
                         when cat = 1 then 'low-medium stress'
                         when cat = 2 then 'medium-high stress'
                         when cat = 3 then 'high stress'
                         when cat = 4 then 'extremely high stress'
                         when cat = 5 then 'arid and low water use'
                    END as label
             FROM bws_rank),
     bsv as (SELECT r.crop, r.irrigation, r.area, r.prod, r.iso, r.name_cntr as region, 'seasonal_variability' as indicator, 2014 as year,
                    coalesce(sev_score, -9999) as  value, coalesce(sev_cat, -9999) as  cat
		     FROM water_risk_indicators_annual_all s
		     LEFT JOIN crops_baseline r on s.aq30_id=r.aq30_id
		     WHERE s.pfaf_id != -9999 and s.gid_1 != '-9999' and  sev_score != -9999 and  sev_cat != -9999),                                                         
bsv_rank as (SELECT sum(value*area)/sum(area) as value, round(sum(cat*area)/sum(area)) as cat, crop, irrigation, iso, region, indicator, year
             FROM bsv
             GROUP BY crop, irrigation, iso, region, indicator, year),
bsv_label as(SELECT value, crop, irrigation, iso, region, indicator, year,
                    CASE when cat = 0 then 'low stress'
                         when cat = 1 then 'low-medium stress'
                         when cat = 2 then 'medium-high stress'
                         when cat = 3 then 'high stress'
                         when cat = 4 then 'extremely high stress'
                    END as label
             FROM bsv_rank),                                                               
     drr as (SELECT r.crop, r.irrigation, r.area, r.prod, r.iso, r.name_cntr as region, 'drought_risk' as indicator, 2014 as year,
                    coalesce(drr_score, -9999) as  value, coalesce(drr_cat, -9999) as  cat 
		     FROM water_risk_indicators_annual_all s
		     LEFT JOIN crops_baseline r on s.aq30_id=r.aq30_id
		     WHERE s.pfaf_id != -9999 and s.gid_1 != '-9999' and drr_score != -9999 and drr_cat != -9999),
drr_rank as (SELECT sum(value*area)/sum(area) as value, round(sum(cat*area)/sum(area)) as cat, crop, irrigation, iso, region, indicator, year
             FROM drr
             GROUP BY crop, irrigation, iso, region, indicator, year),
drr_label as(SELECT value, crop, irrigation, iso, region, indicator, year,
                    CASE when cat = 0 then 'low stress'
                         when cat = 1 then 'low-medium stress'
                         when cat = 2 then 'medium stress'
                         when cat = 3 then 'medium-high stress'
                         when cat = 4 then 'high stress'
                    END as label
             FROM drr_rank),                                                              
     pws as (SELECT r.crop, r.irrigation, r.area, r.prod, r.iso, r.name_cntr as region, indicator, year, label, 
                    CASE when label IN ('Low (<10%)') then 0 
                         when label IN ('Low-medium (10-20%)') then 1 
                         when label IN ('Medium-high (20-40%)') then 2 
                         when label IN ('High (40-80%)') then 3 
                         when label IN ('Extremely high (>80%)') then 4  
                         when label IN ('Arid and low water use') then 5 
                         when label IN ('No data') then -9999 
             END as value
             FROM water_risk_indicators_projections s
             LEFT JOIN crops_projected r on s.basinid=r.basinid
             WHERE type = 'future_value' and indicator = 'water_stress' and scenario = 'business_as_usual' and label != 'No data'),
pws_rank as (SELECT sum(value*area)/sum(area) as value, round(sum(value*area)/sum(area)) as cat, crop, irrigation, iso, region, indicator, year
             FROM pws
             GROUP BY crop, irrigation, iso, region, indicator, year),
pws_label as(SELECT value, crop, irrigation, iso, region, indicator, year,
                    CASE when cat = 0 then 'low stress'
                         when cat = 1 then 'low-medium stress'
                         when cat = 2 then 'medium-high stress'
                         when cat = 3 then 'high stress'
                         when cat = 4 then 'extremely high stress'
                    END as label
             FROM pws_rank),                                                               
     psv as (SELECT r.crop, r.irrigation, r.area, r.prod, r.iso, r.name_cntr as region, indicator, year, label, 
                    CASE when label IN ('Low (<0.33)') then 0 
                         when label IN ('Low-medium (0.33-0.66)') then 1 
                         when label IN ('Medium-high (0.66-1.0)') then 2 
                         when label IN ('High (1.0-1.33)') then 3 
                         when label IN ('Extremely High (>1.33)') then 4  
                         when label IN ('No data') then -9999 
             END as value
             FROM water_risk_indicators_projections s
             LEFT JOIN crops_projected r on s.basinid=r.basinid
             WHERE type = 'future_value' and indicator = 'seasonal_variability' and scenario = 'business_as_usual' and label != 'No data'),
psv_rank as (SELECT sum(value*area)/sum(area) as value, round(sum(value*area)/sum(area)) as cat, crop, irrigation, iso, region, indicator, year
             FROM psv
             GROUP BY crop, irrigation, iso, region, indicator, year),
psv_label as(SELECT value, crop, irrigation, iso, region, indicator, year,
                    CASE when cat = 0 then 'low stress'
                         when cat = 1 then 'low-medium stress'
                         when cat = 2 then 'medium-high stress'
                         when cat = 3 then 'high stress'
                         when cat = 4 then 'extremely high stress'
                    END as label
             FROM psv_rank)                                                                
             
SELECT *  FROM bws_label
UNION ALL
SELECT *  FROM bsv_label  
UNION ALL
SELECT *  FROM drr_label    
UNION ALL
SELECT *  FROM pws_label
UNION ALL
SELECT *  FROM psv_label  
```

In [90]:
payload = {'q': "SELECT * FROM crops_risk_rankings"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df.head()

<Response [200]>


,cartodb_id,crop,indicator,irrigation,iso,label,region,the_geom,the_geom_webmercator,value,year
0,1,arabica coffee,water_stress,irrigated,BDI,low stress,Burundi,NaN,NaN,0.454741,2014
1,2,arabica coffee,water_stress,irrigated,BRA,low stress,Brazil,NaN,NaN,0.626783,2014
2,3,arabica coffee,water_stress,irrigated,COL,low stress,Colombia,NaN,NaN,0.119578,2014
3,4,arabica coffee,water_stress,irrigated,CRI,low-medium stress,Costa Rica,NaN,NaN,1.123613,2014
4,5,arabica coffee,water_stress,irrigated,ECU,low stress,Ecuador,NaN,NaN,0.506026,2014


`url: https://wri-rw.carto.com/api/v2/sql?q=`

`Sql:`
```sql
with alpha as (SELECT crop, year, value
               FROM crops_risk_rankings 
               WHERE indicator='seasonal_variability' and irrigation='rainfed' 
               ORDER BY year asc), 
     betha as (SELECT value, year,
                      CASE when crop IN ('barley','maize','small millet', 'pearl millet','rice','sorghum','wheat','other cereals') then 'Cereals' 
                           when crop IN ('bean', 'cowpea', 'chickpea', 'pigeonpea', 'lentil', 'other pulses') then 'Pulses and legumes' 
                           when crop IN ('cassava','potato','sweet potato','yams', 'other roots') then 'Roots and tubers' 
                           when crop IN ('banana','plantain', 'tropical fruit', 'temperate fruit') then 'Fruit and nuts' 
                           when crop IN ('soybean', 'groundnut', 'coconut', 'oilpalm', 'sunflower', 'rapeseed', 'sesameseed', 'other oil crops') then 'Oilseed crops'  
                           when crop IN ('sugarcane', 'sugarbeet') then 'Sugar crops' 
                           when crop IN ('cotton', 'other fibre crops') then 'Fibres'  
                           when crop IN ('arabica coffee', 'robusta coffee', 'cocoa', 'tea', 'tobacco') then 'Stimulants'  
                           when crop IN ('vegetables') then 'Vegetables' 
                           when crop IN ('rest of crops') then 'Other crops' 
                      END as grupo 
               FROM alpha 
               ORDER BY year asc, grupo)
               
SELECT avg(value) as water_risk, year, grupo as cat 
FROM betha 
WHERE grupo is not null 
GROUP BY grupo, year 
ORDER BY year, cat
```

In [93]:
payload = {'q': "with alpha as (SELECT crop, year, value FROM crops_risk_rankings WHERE indicator='seasonal_variability' and irrigation='rainfed' ORDER BY year asc), betha as (SELECT value, year, CASE when crop IN ('barley','maize','small millet', 'pearl millet','rice','sorghum','wheat','other cereals') then 'Cereals' when crop IN ('bean', 'cowpea', 'chickpea', 'pigeonpea', 'lentil', 'other pulses') then 'Pulses and legumes' when crop IN ('cassava','potato','sweet potato','yams', 'other roots') then 'Roots and tubers' when crop IN ('banana','plantain', 'tropical fruit', 'temperate fruit') then 'Fruit and nuts' when crop IN ('soybean', 'groundnut', 'coconut', 'oilpalm', 'sunflower', 'rapeseed', 'sesameseed', 'other oil crops') then 'Oilseed crops'  when crop IN ('sugarcane', 'sugarbeet') then 'Sugar crops' when crop IN ('cotton', 'other fibre crops') then 'Fibres'  when crop IN ('arabica coffee', 'robusta coffee', 'cocoa', 'tea', 'tobacco') then 'Stimulants'  when crop IN ('vegetables') then 'Vegetables' when crop IN ('rest of crops') then 'Other crops' END as grupo FROM alpha ORDER BY year asc, grupo) SELECT avg(value) as water_risk, year, grupo as cat FROM betha WHERE grupo is not null GROUP BY grupo, year ORDER BY year, cat"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df[df['year'] == 2014]

<Response [200]>


,cat,water_risk,year
0,Cereals,1.893130,2014
1,Fibres,2.095604,2014
2,Fruit and nuts,1.729593,2014
3,Oilseed crops,1.825166,2014
4,Other crops,1.806151,2014
5,Pulses and legumes,1.960893,2014
6,Roots and tubers,1.719248,2014
7,Stimulants,1.643627,2014
8,Sugar crops,1.600214,2014
9,Vegetables,1.667647,2014


#### Percent of crop area by level of water stress and volume of demand
* **Widget_id:** [7b29a2a5-515a-4e22-846f-03ad3d2c0a13](https://staging-api.globalforestwatch.org/v1/widget/7b29a2a5-515a-4e22-846f-03ad3d2c0a13)

**Old query:**
    
`url: https://wri-rw.carto.com/api/v2/sql?q=`

`Sql:`
```sql
with t as (SELECT sum(area) as area, {{water_column}} as risk, crop 
           FROM crops_stats 
           WHERE irrigation='irrigated' 
           GROUP BY {{water_column}}, crop 
           ORDER BY risk asc), 
     r as (SELECT sum(value) as value, commodity 
           FROM combined01_prepared 
           WHERE  commodity in ('Maize','Rice','Soybean','Wheat') and scenario = 'SSP2-MIRO' and region='World' and year='{{year}}' 
                  and impactparameter='Food Demand'  
           GROUP BY commodity), 
     d as (SELECT value, commodity 
           FROM r ) 
           
SELECT crop, risk,value, (area*100/sum(area) over(partition by crop)) area_perc 
FROM d 
INNER JOIN t on lower(commodity)=crop 
ORDER BY risk asc, crop asc
```

`params_config:`
```json
"params_config": [
    {
        "required": true,
        "sufix": "l",
        "key": "water_column"
    },
    {
        "required": true,
        "key": "year"
    },
    {
        "required": true,
        "key": "iso"
    }
],
```

In [10]:
payload = {'q': "with t as (select sum(area) as area, ws4028tl as risk, crop from crops_stats where irrigation='irrigated' group by ws4028tl, crop order by risk asc), r as (SELECT sum(value) as value, commodity FROM combined01_prepared WHERE  commodity in ('Maize','Rice','Soybean','Wheat') and scenario = 'SSP2-MIRO' and region='World' and year=2020 and impactparameter='Food Demand'  group by commodity ), d as ( select value, commodity from r ) select crop, risk,value, (area*100/sum(area) over(partition by crop)) area_perc from d inner join t on lower(commodity)=crop order by risk asc, crop asc"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df

<Response [200]>


,area_perc,crop,risk,value
0,0.084158,maize,1.No Data,132734.593750
1,0.088294,rice,1.No Data,372386.968750
2,0.306822,soybean,1.No Data,18094.544922
3,0.083799,wheat,1.No Data,502489.718750
4,24.664719,maize,2.Low,132734.593750
5,37.776647,rice,2.Low,372386.968750
6,47.706585,soybean,2.Low,18094.544922
7,14.610906,wheat,2.Low,502489.718750
8,9.836103,maize,3.Mid-low,132734.593750
9,12.644658,rice,3.Mid-low,372386.968750


**New query**

- Baseline:

```sql
with a as (SELECT crop, irrigation, iso, area, prod, coalesce(NULLIF(bws_label,''), 'No Data') as risk
           FROM crops_baseline a 
           LEFT JOIN water_risk_indicators_annual b ON b.aq30_id = a.aq30_id
           WHERE crop in ('maize','rice','soybean','wheat')),
     t as (SELECT sum(area) as area, risk, crop 
           FROM a 
           WHERE irrigation='irrigated' 
           GROUP BY risk, crop 
           ORDER BY risk asc), 
     r as (SELECT sum(value) as value, commodity 
           FROM combined01_prepared_new 
           WHERE  commodity in ('maize','rice','soybean','wheat') and scenario = 'SSP2-MIRO' and region='World' and year='{{year}}' 
                  and impactparameter='Food Demand'  
           GROUP BY commodity), 
     d as (SELECT value, commodity 
           FROM r ) 
           
SELECT crop, risk, value, (area*100/sum(area) over(partition by crop)) area_perc 
FROM d 
INNER JOIN t on lower(commodity)=crop 
ORDER BY risk asc, crop asc
```

- Projected:

```sql
with a as (SELECT crop, irrigation, iso, area, prod, label as risk
           FROM crops_projected a 
           LEFT JOIN water_risk_indicators_projections b ON b.basinid = a.basinid
           WHERE crop in ('maize','rice','soybean','wheat') and b.type = 'future_value' and indicator = 'water_stress' and year = 2020 and scenario = 'business_as_usual'),
     t as (SELECT sum(area) as area, risk, crop 
           FROM a 
           WHERE irrigation='irrigated' 
           GROUP BY risk, crop 
           ORDER BY risk asc), 
     r as (SELECT sum(value) as value, commodity 
           FROM combined01_prepared_new 
           WHERE  commodity in ('maize','rice','soybean','wheat') and scenario = 'SSP2-MIRO' and region='World' and year= 2020 
                  and impactparameter='Food Demand'  
           GROUP BY commodity), 
     d as (SELECT value, commodity 
           FROM r ) 
           
SELECT crop, risk,value, (area*100/sum(area) over(partition by crop)) area_perc 
FROM d 
INNER JOIN t on lower(commodity)=crop 
ORDER BY risk asc, crop asc
```

- Combined

First we create an intermediate table called `crops_risk_stats`:
```sql
with a_bws as (SELECT crop, irrigation, iso, name_cntr as country, area, prod, coalesce(NULLIF(bws_label,''), 'No Data') as risk, 'water_stress' as indicator, 
                      2010 as year
               FROM crops_baseline a 
               LEFT JOIN water_risk_indicators_annual b ON b.aq30_id = a.aq30_id),
     b_bws as (SELECT sum(area) as area, sum(prod) as prod, crop, irrigation, iso, country, risk, indicator, year 
               FROM a_bws 
               GROUP BY crop, irrigation, iso, country, risk, indicator, year 
               ORDER BY risk asc),
     a_bwd as (SELECT crop, irrigation, iso, name_cntr as country, area, prod, coalesce(NULLIF(bwd_label,''), 'No Data') as risk, 'water_depletion' as indicator, 
                      2010 as year
               FROM crops_baseline a 
               LEFT JOIN water_risk_indicators_annual b ON b.aq30_id = a.aq30_id),
     b_bwd as (SELECT sum(area) as area, sum(prod) as prod, crop, irrigation, iso, country, risk, indicator, year 
               FROM a_bwd 
               GROUP BY crop, irrigation, iso, country, risk, indicator, year 
               ORDER BY risk asc),
     a_gtd as (SELECT crop, irrigation, iso, name_cntr as country, area, prod, coalesce(NULLIF(gtd_label,''), 'No Data') as risk, 
                      'groundwater_table_decline' as indicator, 2010 as year
               FROM crops_baseline a 
               LEFT JOIN water_risk_indicators_annual b ON b.aq30_id = a.aq30_id),
     b_gtd as (SELECT sum(area) as area, sum(prod) as prod, crop, irrigation, iso, country, risk, indicator, year 
               FROM a_gtd 
               GROUP BY crop, irrigation, iso, country, risk, indicator, year 
               ORDER BY risk asc),
     a_iav as (SELECT crop, irrigation, iso, name_cntr as country, area, prod, coalesce(NULLIF(iav_label,''), 'No Data') as risk, 
                      'interannual_variability' as indicator, 2010 as year
               FROM crops_baseline a 
               LEFT JOIN water_risk_indicators_annual b ON b.aq30_id = a.aq30_id),
     b_iav as (SELECT sum(area) as area, sum(prod) as prod, crop, irrigation, iso, country, risk, indicator, year 
               FROM a_iav 
               GROUP BY crop, irrigation, iso, country, risk, indicator, year 
               ORDER BY risk asc),
     a_sev as (SELECT crop, irrigation, iso, name_cntr as country, area, prod, coalesce(NULLIF(sev_label,''), 'No Data') as risk, 
                      'seasonal_variability' as indicator, 2010 as year
               FROM crops_baseline a 
               LEFT JOIN water_risk_indicators_annual b ON b.aq30_id = a.aq30_id),
     b_sev as (SELECT sum(area) as area, sum(prod) as prod, crop, irrigation, iso, country, risk, indicator, year 
               FROM a_sev 
               GROUP BY crop, irrigation, iso, country, risk, indicator, year 
               ORDER BY risk asc),
     a_drr as (SELECT crop, irrigation, iso, name_cntr as country, area, prod, coalesce(NULLIF(drr_label,''), 'No Data') as risk, 
                      'drought_risk' as indicator, 2010 as year
               FROM crops_baseline a 
               LEFT JOIN water_risk_indicators_annual b ON b.aq30_id = a.aq30_id),
     b_drr as (SELECT sum(area) as area, sum(prod) as prod, crop, irrigation, iso, country, risk, indicator, year 
               FROM a_drr
               GROUP BY crop, irrigation, iso, country, risk, indicator, year 
               ORDER BY risk asc),
     a_rfr as (SELECT crop, irrigation, iso, name_cntr as country, area, prod, coalesce(NULLIF(rfr_label,''), 'No Data') as risk, 
                      'riverine_flood_risk' as indicator, 2010 as year
               FROM crops_baseline a 
               LEFT JOIN water_risk_indicators_annual b ON b.aq30_id = a.aq30_id),
     b_rfr as (SELECT sum(area) as area, sum(prod) as prod, crop, irrigation, iso, country, risk, indicator, year 
               FROM a_rfr
               GROUP BY crop, irrigation, iso, country, risk, indicator, year 
               ORDER BY risk asc),
     a_cfr as (SELECT crop, irrigation, iso, name_cntr as country, area, prod, coalesce(NULLIF(cfr_label,''), 'No Data') as risk, 
                      'coastal_flood_risk' as indicator, 2010 as year
               FROM crops_baseline a 
               LEFT JOIN water_risk_indicators_annual b ON b.aq30_id = a.aq30_id),
     b_cfr as (SELECT sum(area) as area, sum(prod) as prod, crop, irrigation, iso, country, risk, indicator, year 
               FROM a_cfr
               GROUP BY crop, irrigation, iso, country, risk, indicator, year 
               ORDER BY risk asc),
     a_ucw as (SELECT crop, irrigation, iso, name_cntr as country, area, prod, coalesce(NULLIF(ucw_label,''), 'No Data') as risk, 
                      'untreated_collected_wastewater' as indicator, 2010 as year
               FROM crops_baseline a 
               LEFT JOIN water_risk_indicators_annual b ON b.aq30_id = a.aq30_id),
     b_ucw as (SELECT sum(area) as area, sum(prod) as prod, crop, irrigation, iso, country, risk, indicator, year 
               FROM a_ucw
               GROUP BY crop, irrigation, iso, country, risk, indicator, year 
               ORDER BY risk asc),
     a_cep as (SELECT crop, irrigation, iso, name_cntr as country, area, prod, coalesce(NULLIF(cep_label,''), 'No Data') as risk, 
                      'coastal_eutrophication_potential' as indicator, 2010 as year
               FROM crops_baseline a 
               LEFT JOIN water_risk_indicators_annual b ON b.aq30_id = a.aq30_id),
     b_cep as (SELECT sum(area) as area, sum(prod) as prod, crop, irrigation, iso, country, risk, indicator, year 
               FROM a_cep
               GROUP BY crop, irrigation, iso, country, risk, indicator, year 
               ORDER BY risk asc),
     a_udw as (SELECT crop, irrigation, iso, name_cntr as country, area, prod, coalesce(NULLIF(udw_label,''), 'No Data') as risk, 
                      'unimproved_drinking_water' as indicator, 2010 as year
               FROM crops_baseline a 
               LEFT JOIN water_risk_indicators_annual b ON b.aq30_id = a.aq30_id),
     b_udw as (SELECT sum(area) as area, sum(prod) as prod, crop, irrigation, iso, country, risk, indicator, year 
               FROM a_udw
               GROUP BY crop, irrigation, iso, country, risk, indicator, year 
               ORDER BY risk asc),
     a_usa as (SELECT crop, irrigation, iso, name_cntr as country, area, prod, coalesce(NULLIF(usa_label,''), 'No Data') as risk, 
                      'unimproved_sanitation' as indicator, 2010 as year
               FROM crops_baseline a 
               LEFT JOIN water_risk_indicators_annual b ON b.aq30_id = a.aq30_id),
     b_usa as (SELECT sum(area) as area, sum(prod) as prod, crop, irrigation, iso, country, risk, indicator, year 
               FROM a_usa
               GROUP BY crop, irrigation, iso, country, risk, indicator, year 
               ORDER BY risk asc),
     a_rri as (SELECT crop, irrigation, iso, name_cntr as country, area, prod, coalesce(NULLIF(rri_label,''), 'No Data') as risk, 
                      'reprisk_index' as indicator, 2010 as year
               FROM crops_baseline a 
               LEFT JOIN water_risk_indicators_annual b ON b.aq30_id = a.aq30_id),
     b_rri as (SELECT sum(area) as area, sum(prod) as prod, crop, irrigation, iso, country, risk, indicator, year 
               FROM a_rri
               GROUP BY crop, irrigation, iso, country, risk, indicator, year 
               ORDER BY risk asc),
         c_ws as (SELECT crop, irrigation, iso, name_cntr as country, area, prod, label as risk, indicator, year
               FROM crops_projected a 
               LEFT JOIN water_risk_indicators_projections b ON b.basinid = a.basinid
               WHERE b.type = 'future_value' and indicator = 'water_stress' and scenario = 'business_as_usual'),
         d_ws as (SELECT sum(area) as area, sum(prod) as prod, crop, irrigation, iso, country, risk, indicator, year 
               FROM c_ws
               GROUP BY risk, crop, irrigation, iso, country, risk, indicator, year 
               ORDER BY risk asc),
         c_sv as (SELECT crop, irrigation, iso, name_cntr as country, area, prod, label as risk, indicator, year
               FROM crops_projected a 
               LEFT JOIN water_risk_indicators_projections b ON b.basinid = a.basinid
               WHERE b.type = 'future_value' and indicator = 'seasonal_variability' and scenario = 'business_as_usual'),
         d_sv as (SELECT sum(area) as area, sum(prod) as prod, crop, irrigation, iso, country, risk, indicator, year 
               FROM c_sv
               GROUP BY risk, crop, irrigation, iso, country, risk, indicator, year 
               ORDER BY risk asc)

SELECT *  FROM b_bws 
UNION ALL
SELECT *  FROM b_bwd 
UNION ALL
SELECT *  FROM b_gtd 
UNION ALL
SELECT *  FROM b_iav 
UNION ALL
SELECT *  FROM b_sev 
UNION ALL
SELECT *  FROM b_drr 
UNION ALL
SELECT *  FROM b_rfr 
UNION ALL
SELECT *  FROM b_cfr 
UNION ALL
SELECT *  FROM b_ucw 
UNION ALL
SELECT *  FROM b_cep 
UNION ALL
SELECT *  FROM b_udw 
UNION ALL
SELECT *  FROM b_usa 
UNION ALL
SELECT *  FROM b_rri 
UNION ALL
SELECT *  FROM d_ws
UNION ALL
SELECT *  FROM d_sv
```

In [70]:
payload = {'q': "SELECT * FROM crops_risk_stats"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df.head()

<Response [200]>


,area,cartodb_id,country,crop,indicator,irrigation,iso,prod,risk,the_geom,the_geom_webmercator,year
0,265.4,1,Ethiopia,arabica coffee,water_stress,irrigated,ETH,96.8,Arid and Low Water Use,NaN,NaN,2010
1,115.7,2,Yemen,arabica coffee,water_stress,irrigated,YEM,76.7,Arid and Low Water Use,NaN,NaN,2010
2,0.2,3,Bolivia,arabica coffee,water_stress,rainfed,BOL,0.2,Arid and Low Water Use,NaN,NaN,2010
3,2108.7,4,Ethiopia,arabica coffee,water_stress,rainfed,ETH,1434.2,Arid and Low Water Use,NaN,NaN,2010
4,72.5,5,Yemen,arabica coffee,water_stress,rainfed,YEM,21.4,Arid and Low Water Use,NaN,NaN,2010


`url: https://wri-rw.carto.com/api/v2/sql?q=`

`Sql:`
```sql
with t as (SELECT sum(area) as area, initcap(split_part(risk, ' (', 1)) as risk, crop 
           FROM crops_risk_stats
           WHERE indicator='water_stress' and irrigation='irrigated' and crop in ('maize','rice','soybean','wheat') and year = {{year}}
           GROUP BY risk, crop), 
     r as (SELECT sum(value) as value, commodity 
           FROM combined01_prepared 
           WHERE  commodity in ('Maize','Rice','Soybean','Wheat') and scenario = 'SSP2-MIRO' and region='World' and year = {{year}}
                  and impactparameter='Food Demand'  
           GROUP BY commodity), 
     d as (SELECT value, commodity 
           FROM r ) 
           
SELECT crop, value, (area*100/sum(area) over(partition by crop)) area_perc, 
       CASE when risk IN ('No Data') then '1.No Data'
			when risk IN ('Low') then '2.Low'
			when risk IN ('Low - Medium') then '3.Low-Mid'
			when risk IN ('Low-Medium') then '3.Low-Mid'                                            
			when risk IN ('Medium - High') then '4.Mid-High'
			when risk IN ('Medium-High') then '4.Mid-High'                                          
			when risk IN ('High') then '5.High'
			when risk IN ('Extremely High') then '6.Extremely High'
        END as risk
FROM d 
INNER JOIN t on lower(commodity)=crop 
ORDER BY risk asc, crop asc                                                  
```

`params_config:`
```json
"params_config": [
    {
        "required": true,
        "key": "year"
    }
],
```

In [39]:
payload = {'q': "with t as (SELECT sum(area) as area, initcap(split_part(risk, ' (', 1)) as risk, crop FROM crops_risk_stats WHERE indicator='water_stress' and irrigation='irrigated' and crop in ('maize','rice','soybean','wheat') and year = 2010 GROUP BY risk, crop), r as (SELECT sum(value) as value, commodity FROM combined01_prepared WHERE  commodity in ('Maize','Rice','Soybean','Wheat') and scenario = 'SSP2-MIRO' and region='World' and year = 2010 and impactparameter='Food Demand' GROUP BY commodity), d as (SELECT value, commodity FROM r ) SELECT crop, value, (area*100/sum(area) over(partition by crop)) area_perc, CASE when risk IN ('No Data') then '1.No Data' when risk IN ('Low') then '2.Low' when risk IN ('Low - Medium') then '3.Low-Mid' when risk IN ('Low-Medium') then '3.Low-Mid' when risk IN ('Medium - High') then '4.Mid-High' when risk IN ('Medium-High') then '4.Mid-High' when risk IN ('High') then '5.High' when risk IN ('Extremely High') then '6.Extremely High' END as risk FROM d INNER JOIN t on lower(commodity)=crop ORDER BY risk asc, crop asc "}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df

<Response [200]>


,area_perc,crop,risk,value
0,0.031596,maize,1.No Data,115050.101562
1,0.100011,rice,1.No Data,350160.468750
2,0.041433,soybean,1.No Data,12415.016602
3,0.013948,wheat,1.No Data,447774.968750
4,16.911364,maize,2.Low,115050.101562
5,47.760585,rice,2.Low,350160.468750
6,35.791121,soybean,2.Low,12415.016602
7,11.593440,wheat,2.Low,447774.968750
8,6.588350,maize,3.Low-Mid,115050.101562
9,11.039626,rice,3.Low-Mid,350160.468750


#### Average global change in water stress for irrigated crops through 2040
* **Widget_id:** [4f322f4a-dde6-4f63-b7c9-10b0ba0c1402](https://staging-api.globalforestwatch.org/v1/widget/4f322f4a-dde6-4f63-b7c9-10b0ba0c1402)

**Old query:**
    
`url: https://wri-rw.carto.com/api/v2/sql?q=`

`Sql:`
```sql
with alpha as (SELECT crop, year, value
               FROM crops_risk_rankings 
               WHERE indicator='water_stress' and irrigation='irrigated' 
               ORDER BY year asc), 
     betha as (SELECT value, year,
                      CASE when crop IN ('barley','maize','small millet', 'pearl millet','rice','sorghum','wheat','other cereals') then 'Cereals' 
                           when crop IN ('bean', 'cowpea', 'chickpea', 'pigeonpea', 'lentil', 'other pulses') then 'Pulses and legumes' 
                           when crop IN ('cassava','potato','sweet potato','yams', 'other roots') then 'Roots and tubers' 
                           when crop IN ('banana','plantain', 'tropical fruit', 'temperate fruit') then 'Fruit and nuts' 
                           when crop IN ('soybean', 'groundnut', 'coconut', 'oilpalm', 'sunflower', 'rapeseed', 'sesameseed', 'other oil crops') then 'Oilseed crops'  
                           when crop IN ('sugarcane', 'sugarbeet') then 'Sugar crops' 
                           when crop IN ('cotton', 'other fibre crops') then 'Fibres'  
                           when crop IN ('arabica coffee', 'robusta coffee', 'cocoa', 'tea', 'tobacco') then 'Stimulants'  
                           when crop IN ('vegetables') then 'Vegetables' 
                           when crop IN ('rest of crops') then 'Other crops' 
                      END as grupo 
               FROM alpha 
               ORDER BY year asc, grupo)

SELECT avg(value) as water_risk, year, grupo as cat 
FROM betha 
WHERE grupo is not null and year in (2014, 2030, 2040)
GROUP BY grupo, year 
ORDER BY year, 
CASE grupo WHEN 'Cereals' THEN 1 
                             WHEN 'Pulses and legumes' THEN 2 
                             WHEN 'Roots and tubers' THEN 3 
                             WHEN 'Fruit and nuts' THEN 4 
                             WHEN 'Oilseed crops' THEN 5
                             WHEN 'Sugar crops' THEN 6 
                             WHEN 'Fibres' THEN 7
                             WHEN 'Stimulants' THEN 8
                             WHEN 'Vegetables' THEN 9
                             WHEN 'Other crops' THEN 10
                             ELSE 11 END
```

In [87]:
payload = {'q': "with alpha as (SELECT crop, year, value FROM crops_risk_rankings WHERE indicator='water_stress' and irrigation='irrigated' ORDER BY year asc), betha as (SELECT value, year, CASE when crop IN ('barley','maize','small millet', 'pearl millet','rice','sorghum','wheat','other cereals') then 'Cereals' when crop IN ('bean', 'cowpea', 'chickpea', 'pigeonpea', 'lentil', 'other pulses') then 'Pulses and legumes' when crop IN ('cassava','potato','sweet potato','yams', 'other roots') then 'Roots and tubers' when crop IN ('banana','plantain', 'tropical fruit', 'temperate fruit') then 'Fruit and nuts' when crop IN ('soybean', 'groundnut', 'coconut', 'oilpalm', 'sunflower', 'rapeseed', 'sesameseed', 'other oil crops') then 'Oilseed crops'  when crop IN ('sugarcane', 'sugarbeet') then 'Sugar crops' when crop IN ('cotton', 'other fibre crops') then 'Fibres'  when crop IN ('arabica coffee', 'robusta coffee', 'cocoa', 'tea', 'tobacco') then 'Stimulants'  when crop IN ('vegetables') then 'Vegetables' when crop IN ('rest of crops') then 'Other crops' END as grupo FROM alpha ORDER BY year asc, grupo) SELECT avg(value) as water_risk, year, grupo as cat FROM betha WHERE grupo is not null and year in (2014, 2030, 2040) GROUP BY grupo, year ORDER BY year, CASE grupo WHEN 'Cereals' THEN 1 WHEN 'Pulses and legumes' THEN 2 WHEN 'Roots and tubers' THEN 3 WHEN 'Fruit and nuts' THEN 4 WHEN 'Oilseed crops' THEN 5 WHEN 'Sugar crops' THEN 6 WHEN 'Fibres' THEN 7 WHEN 'Stimulants' THEN 8 WHEN 'Vegetables' THEN 9 WHEN 'Other crops' THEN 10 ELSE 11 END"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df

<Response [200]>


,cat,water_risk,year
0,Cereals,2.189740,2014
1,Pulses and legumes,2.569900,2014
2,Roots and tubers,2.073402,2014
3,Fruit and nuts,2.123900,2014
4,Oilseed crops,2.284048,2014
5,Sugar crops,1.648568,2014
6,Fibres,1.908992,2014
7,Stimulants,1.661184,2014
8,Vegetables,1.770574,2014
9,Other crops,2.028542,2014


**New query**

`url: https://wri-rw.carto.com/api/v2/sql?q=`

`Sql:`
```sql
with alpha as (SELECT crop, year, value
               FROM crops_risk_rankings 
               WHERE indicator='water_stress' and irrigation='irrigated' 
               ORDER BY year asc), 
     betha as (SELECT value, year,
                      CASE when crop IN ('barley','maize','small millet', 'pearl millet','rice','sorghum','wheat','other cereals') then 'Cereals' 
                           when crop IN ('bean', 'cowpea', 'chickpea', 'pigeonpea', 'lentil', 'other pulses') then 'Pulses and legumes' 
                           when crop IN ('cassava','potato','sweet potato','yams', 'other roots') then 'Roots and tubers' 
                           when crop IN ('banana','plantain', 'tropical fruit', 'temperate fruit') then 'Fruit and nuts' 
                           when crop IN ('soybean', 'groundnut', 'coconut', 'oilpalm', 'sunflower', 'rapeseed', 'sesameseed', 'other oil crops') then 'Oilseed crops'  
                           when crop IN ('sugarcane', 'sugarbeet') then 'Sugar crops' 
                           when crop IN ('cotton', 'other fibre crops') then 'Fibres'  
                           when crop IN ('arabica coffee', 'robusta coffee', 'cocoa', 'tea', 'tobacco') then 'Stimulants'  
                           when crop IN ('vegetables') then 'Vegetables' 
                           when crop IN ('rest of crops') then 'Other crops' 
                      END as grupo 
               FROM alpha 
               ORDER BY year asc, grupo)
               
SELECT avg(value) as water_risk, year, grupo as cat 
FROM betha 
WHERE grupo is not null 
GROUP BY grupo, year 
ORDER BY year, cat
```

In [85]:
payload = {'q': "with alpha as (SELECT crop, year, value FROM crops_risk_rankings WHERE indicator='water_stress' and irrigation='irrigated' ORDER BY year asc), betha as (SELECT value, year, CASE when crop IN ('barley','maize','small millet', 'pearl millet','rice','sorghum','wheat','other cereals') then 'Cereals' when crop IN ('bean', 'cowpea', 'chickpea', 'pigeonpea', 'lentil', 'other pulses') then 'Pulses and legumes' when crop IN ('cassava','potato','sweet potato','yams', 'other roots') then 'Roots and tubers' when crop IN ('banana','plantain', 'tropical fruit', 'temperate fruit') then 'Fruit and nuts' when crop IN ('soybean', 'groundnut', 'coconut', 'oilpalm', 'sunflower', 'rapeseed', 'sesameseed', 'other oil crops') then 'Oilseed crops'  when crop IN ('sugarcane', 'sugarbeet') then 'Sugar crops' when crop IN ('cotton', 'other fibre crops') then 'Fibres'  when crop IN ('arabica coffee', 'robusta coffee', 'cocoa', 'tea', 'tobacco') then 'Stimulants'  when crop IN ('vegetables') then 'Vegetables' when crop IN ('rest of crops') then 'Other crops' END as grupo FROM alpha ORDER BY year asc, grupo) SELECT avg(value) as water_risk, year, grupo as cat FROM betha WHERE grupo is not null GROUP BY grupo, year ORDER BY year, cat"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df

<Response [200]>


,cat,water_risk,year
0,Cereals,2.189740,2014
1,Fibres,1.908992,2014
2,Fruit and nuts,2.123900,2014
3,Oilseed crops,2.284048,2014
4,Other crops,2.028542,2014
5,Pulses and legumes,2.569900,2014
6,Roots and tubers,2.073402,2014
7,Stimulants,1.661184,2014
8,Sugar crops,1.648568,2014
9,Vegetables,1.770574,2014


#### Irrigated vs Rainfed

* **Widget_id**: [606c813b-d716-4b0b-8717-9389425c9262](https://staging-api.globalforestwatch.org/v1/widget/606c813b-d716-4b0b-8717-9389425c9262)

**Old query:**

`url: https://wri-01.carto.com/api/v2/sql?=`

`Sql:`
```sql
SELECT sum(value) as value, irrigation as category 
FROM combined01_prepared_new 
WHERE impactparameter='Area' and scenario='SSP2-MIRO' {{and}} and commodity<>'all cereals' and commodity<>'all pulses' and region <> 'World' 
GROUP BY irrigation
```
`sql_config:`
```json
"sql_config": [
    {
        "key_params": [
            {
                "required": true,
                "key": "year"
            },
            {
                "required": false,
                "key": "iso"
            },
            {
                "required": false,
                "key": "commodity"
            }
        ],
        "key": "and"
    }
```

In [25]:
payload = {'q': "SELECT sum(value) as value, irrigation as category FROM combined01_prepared_new where impactparameter='Area' and scenario='SSP2-MIRO' AND year = 2010 and commodity<>'all cereals' and commodity<>'all pulses' and region <> 'World' group by irrigation"}
r = requests.get('https://wri-01.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df

<Response [200]>


,category,value
0,irrigated,248981.097052
1,rainfed,649713.061795


**New query:**

`url: https://wri-rw.carto.com/api/v2/sql?=`

`Sql:`
```sql
SELECT sum(area) as value, irrigation as category
FROM crops_risk_stats
WHERE indicator = 'water_stress' {{and}} 
GROUP BY irrigation 
```
`sql_config:`
```json
"sql_config": [
    {
        "key_params": [
            {
                "required": true,
                "key": "year"
            },
            {
                "required": false,
                "key": "iso"
            },
            {
                "required": false,
                "key": "crop"
            }
        ],
        "key": "and"
    }
```

In [31]:
payload = {'q': "SELECT sum(area) as value, irrigation as category FROM crops_risk_stats WHERE indicator = 'water_stress' and year = 2010 GROUP BY irrigation"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df

<Response [200]>


,category,value
0,irrigated,2.735708e+08
1,rainfed,1.008788e+09


#### Percentage of crop production area by water risk in year
* **Widget_id:** [7b37bd6e-a465-4c4d-98b3-1a110a66d27e](https://staging-api.globalforestwatch.org/v1/widget/7b37bd6e-a465-4c4d-98b3-1a110a66d27e)

**Old query:**

`url: https://wri-rw.carto.com/api/v2/sql?=`

`Sql:`
```sql
with ff as (select *, case when {{year}}=2010  then  2010 else year  end as y  
            from water_risk_indicators 
            where (year={{year}} or year = 2014)),  
     wr as (SELECT basinid, indicator, label, value 
            FROM ff 
            where type='absolute' and period='year' and y={{year}} and indicator in ('seasonal_variability', 'drought_severity', 'water_stress','groundwater_stress', 'interannual_variability') and label is not null 
                  and label !='' and label !='-32767'), 
   prod as (SELECT sum(value) as value 
            FROM combined01_prepared 
            where year={{year}} and  commodity<>'All Cereals' and commodity<>'All Pulses' and impactparameter='Production' and scenario = 'SSP2-MIRO' 
            order by value desc),   
      s as (SELECT sum(area) as perc, min(basin_total_area) b_area, basinid, irrigation 
            FROM all_crops_stats 
            group by basinid, irrigation), 
      f as (select  wr.basinid, indicator, label, perc, b_area, irrigation,sum(perc) over (partition by indicator) as sdss, value  
            from s 
            inner join wr on wr.basinid=s.basinid), 
      t as (select sum(perc)*100/sdss a, label, indicator, irrigation, max(value) as value_m  
            from f 
            group by label, indicator, irrigation, sdss 
            order by indicator asc, label asc ) 
            
 select initcap(split_part(label, ' (', 1)) as risk, row_number() over (partition by indicator order by value_m asc) as index, replace(initcap(indicator), '_', ' ') as indicator, irrigation, prod.value*a as perc_value, value_m, a as area_perc 
from t, prod  
where (indicator='water_stress' and irrigation='irrigated') or (indicator='drought_severity' and irrigation='rainfed') or (indicator='seasonal_variability' and irrigation='rainfed') or (indicator='interannual_variability' and irrigation='irrigated') or (indicator='groundwater_stress' and irrigation='irrigated')
```

`sql_config:`
```json
"sql_config": [
    {
        "key_params": [
            {
                "required": true,
                "key": "year"
            }
        ],
        "key": "where"
    }
],
```

`params_config:`
```json
"params_config": [
    {
        "key": "year",
        "required": true
    },
    {
        "required": true,
        "key": "commodity"
    },
    {
        "required": true,
        "key": "crop"
    }
],
```

In [27]:
payload = {'q': "with ff as (select *, case when 2010=2010  then  2010 else year  end as y from water_risk_indicators where (year=2010 or year = 2014)), wr as (SELECT basinid, indicator, label, value FROM ff where type='absolute' and period='year' and y=2010 and indicator in ('seasonal_variability', 'drought_severity', 'water_stress','groundwater_stress', 'interannual_variability') and label is not null and label !='' and label !='-32767'), prod as (SELECT sum(value) as value FROM combined01_prepared where year=2010 and  commodity<>'All Cereals' and commodity<>'All Pulses' and impactparameter='Production' and scenario = 'SSP2-MIRO' order by value desc), s as (SELECT sum(area) as perc, min(basin_total_area) b_area, basinid, irrigation FROM all_crops_stats group by basinid, irrigation), f as (select  wr.basinid, indicator, label, perc, b_area, irrigation,sum(perc) over (partition by indicator) as sdss, value  from s inner join wr on wr.basinid=s.basinid), t as (select sum(perc)*100/sdss a, label, indicator, irrigation, max(value) as value_m  from f group by label, indicator, irrigation, sdss order by indicator asc, label asc ) select initcap(split_part(label, ' (', 1)) as risk, row_number() over (partition by indicator order by value_m asc) as index, replace(initcap(indicator), '_', ' ') as indicator, irrigation, prod.value*a as perc_value, value_m, a as area_perc from t, prod  where (indicator='water_stress' and irrigation='irrigated') or (indicator='drought_severity' and irrigation='rainfed') or (indicator='seasonal_variability' and irrigation='rainfed') or (indicator='interannual_variability' and irrigation='irrigated') or (indicator='groundwater_stress' and irrigation='irrigated')"}
r = requests.get('https://wri-01.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df

<Response [200]>


,area_perc,index,indicator,irrigation,perc_value,risk,value_m
0,10.851155,1,Drought Severity,rainfed,7.138623e+07,1. Low,1.999961e+01
1,47.039787,2,Drought Severity,rainfed,3.094595e+08,2. Low To Medium,2.999941e+01
2,17.833142,3,Drought Severity,rainfed,1.173184e+08,3. Medium To High,3.999206e+01
3,3.329582,4,Drought Severity,rainfed,2.190424e+07,4. High,4.998952e+01
4,2.073203,5,Drought Severity,rainfed,1.363893e+07,5. Extremely High,2.121970e+02
5,8.925560,1,Groundwater Stress,irrigated,5.871836e+07,No Data,-3.276700e+04
6,3.765796,2,Groundwater Stress,irrigated,2.477395e+07,1. Low,9.970000e-01
7,1.958447,3,Groundwater Stress,irrigated,1.288399e+07,2. Low To Medium,4.877000e+00
8,1.268276,4,Groundwater Stress,irrigated,8.343579e+06,3. Medium To High,9.968000e+00
9,0.522978,5,Groundwater Stress,irrigated,3.440499e+06,4. High,1.966300e+01


**New query:**

`url: https://wri-rw.carto.com/api/v2/sql?=`

`Sql:`
```sql
with a as (SELECT sum(area) as area_perc, indicator, irrigation, sum(prod) as perc_value, risk
           FROM crops_risk_stats
           WHERE year={{year}} 
           GROUP BY indicator, irrigation, risk
           ORDER BY indicator, indicator, risk),
     b as (SELECT initcap(split_part(risk, ' (', 1)) as risk, replace(initcap(indicator), '_', ' ') as indicator, area_perc, perc_value, irrigation
           FROM a
           WHERE (indicator='water_stress' and irrigation='irrigated') or (indicator='seasonal_variability' and irrigation='rainfed') or 
                 (indicator='interannual_variability' and irrigation='irrigated') or 
                 (indicator='coastal_eutrophication_potential' and irrigation='irrigated')),

     c as (SELECT indicator, area_perc, perc_value, irrigation,  
                  CASE when risk IN ('No Data') then '1.No Data'
                       when risk IN ('Low') then '2.Low'
                       when risk IN ('Low - Medium') then '3.Low-Mid'
                       when risk IN ('Low-Medium') then '3.Low-Mid'
                       when risk IN ('Medium - High') then '4.Mid-High'
                       when risk IN ('Medium-High') then '4.Mid-High'
                       when risk IN ('High') then '5. High'
                       when risk IN ('Extremely High') then '6.Extremely High'
                       when risk IN ('Arid And Low Water Use') then 'Arid and Low Water Use'
                  END as risk,
                  CASE when risk IN ('No Data') then -9999
                       when risk IN ('Low') then 0
                       when risk IN ('Low - Medium') then 1
                       when risk IN ('Low-Medium') then 1
                       when risk IN ('Medium - High') then 2
                       when risk IN ('Medium-High') then 2
                       when risk IN ('High') then 3
                       when risk IN ('Extremely High') then 4
                       when risk IN ('Arid And Low Water Use') then 5
                  END as value_m

           FROM b)

SELECT indicator, area_perc, perc_value, irrigation, risk, value_m, row_number() over (partition by indicator order by value_m asc) as index 
FROM c
```


`sql_config:`
```json
"sql_config": [
    {
        "key_params": [
            {
                "required": true,
                "key": "year"
            }
        ],
        "key": "where"
    }
],
```

`params_config:`
```json
"params_config": [
    {
        "key": "year",
        "required": true
    }
],
```

In [5]:
payload = {'q': "with a as (SELECT sum(area) as area_perc, indicator, irrigation, sum(prod) as perc_value, risk FROM crops_risk_stats WHERE year=2010 GROUP BY indicator, irrigation, risk ORDER BY indicator, indicator, risk), b as (SELECT initcap(split_part(risk, ' (', 1)) as risk, replace(initcap(indicator), '_', ' ') as indicator, area_perc, perc_value, irrigation FROM a WHERE (indicator='water_stress' and irrigation='irrigated') or (indicator='seasonal_variability' and irrigation='rainfed') or (indicator='interannual_variability' and irrigation='irrigated') or (indicator='coastal_eutrophication_potential' and irrigation='irrigated')), c as (SELECT indicator, area_perc, perc_value, irrigation, CASE when risk IN ('No Data') then '1.No Data' when risk IN ('Low') then '2.Low' when risk IN ('Low - Medium') then '3.Low-Mid' when risk IN ('Low-Medium') then '3.Low-Mid' when risk IN ('Medium - High') then '4.Mid-High' when risk IN ('Medium-High') then '4.Mid-High' when risk IN ('High') then '5. High' when risk IN ('Extremely High') then '6.Extremely High' when risk IN ('Arid And Low Water Use') then 'Arid and Low Water Use' END as risk, CASE when risk IN ('No Data') then -9999 when risk IN ('Low') then 0 when risk IN ('Low - Medium') then 1 when risk IN ('Low-Medium') then 1 when risk IN ('Medium - High') then 2 when risk IN ('Medium-High') then 2 when risk IN ('High') then 3 when risk IN ('Extremely High') then 4 when risk IN ('Arid And Low Water Use') then 5 END as value_m FROM b) SELECT indicator, area_perc, perc_value, irrigation, risk, value_m, row_number() over (partition by indicator order by value_m asc) as index FROM c"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df

<Response [200]>


,indicator,area_perc,perc_value,irrigation,risk,value_m,index
0,Coastal Eutrophication Potential,216169.9,3.741632e+06,irrigated,1.No Data,-9999,1
1,Coastal Eutrophication Potential,23127353.7,2.870754e+08,irrigated,2.Low,0,2
2,Coastal Eutrophication Potential,62076883.8,6.995646e+08,irrigated,3.Low-Mid,1,3
3,Coastal Eutrophication Potential,84226389.2,6.602767e+08,irrigated,4.Mid-High,2,4
4,Coastal Eutrophication Potential,48359562.8,4.699507e+08,irrigated,5. High,3,5
5,Coastal Eutrophication Potential,55564469.8,4.209524e+08,irrigated,6.Extremely High,4,6
6,Interannual Variability,202290.7,2.922963e+06,irrigated,1.No Data,-9999,1
7,Interannual Variability,60284240.5,5.371749e+08,irrigated,2.Low,0,2
8,Interannual Variability,144107761.8,1.407660e+09,irrigated,3.Low-Mid,1,3
9,Interannual Variability,50302618.6,4.407820e+08,irrigated,4.Mid-High,2,4


***
### COUNTRY
#### Percent of crop area by level of water stress and volume of demand in country
* **Widget_id:** [e2778762-4b3d-498c-8b4a-869ac834f406](https://staging-api.globalforestwatch.org/v1/widget/e2778762-4b3d-498c-8b4a-869ac834f406) 

**Old query:**

`url: https://wri-rw.carto.com/api/v2/sql?=`

`Sql:`
```sql
with ss  as (SELECT * 
             FROM crops_countries_stats where iso = '{{iso}}'), 
      sd as (SELECT ss.*, wsbs00tl as risk 
             FROM ss inner join crops_stats on ss.basinid=crops_stats.basinid and ss.crop=crops_stats.crop), 
       t as (select sum(area) as area, risk, crop 
             from sd 
             group by  risk, crop 
             order by risk asc), 
       r as (SELECT sum(value) as value, commodity 
             FROM combined01_prepared 
             WHERE commodity<>'All Cereals' and commodity<>'All Pulses' and iso='{{iso}}' and year='{{year}}' and impactparameter='Food Demand' and scenario='SSP2-MIRO' group by commodity), 
       d as (select value, commodity 
             from r ) 
             
select crop, risk, value, ((area*100)/sum(area) over(partition by crop)) area_perc 
from d 
inner join t on lower(commodity)=crop 
order by risk asc, crop asc
``` 

`params_config:`
```json
                "params_config": [
                    {
                        "required": true,
                        "key": "year"
                    },
                    {
                        "required": true,
                        "key": "iso"
                    },
                    {
                        "required": true,
                        "key": "countryName"
                    },
                    {
                        "required": true,
                        "key": "crop_name"
                    }
                ],
```

**New query**
`url: https://wri-rw.carto.com/api/v2/sql?q=`

`Sql:`
```sql                                           
with t as (SELECT sum(area) as area, initcap(split_part(risk, ' (', 1)) as risk, crop 
           FROM crops_risk_stats
           WHERE indicator='water_stress' and irrigation='irrigated' and crop in ('maize','rice','soybean','wheat') and year = {{year}} and iso = '{{iso}}'
           GROUP BY risk, crop), 
     r as (SELECT sum(value) as value, commodity 
           FROM combined01_prepared 
           WHERE commodity<>'All Cereals' and commodity<>'All Pulses' and impactparameter='Food Demand' and scenario='SSP2-MIRO' and year = {{year}} and iso = '{{iso}}'
           GROUP BY commodity), 
     d as (SELECT value, commodity 
           FROM r ) 
           
SELECT crop, value, (area*100/sum(area) over(partition by crop)) area_perc, 
       CASE when risk IN ('No Data') then '1.No Data'
			when risk IN ('Low') then '2.Low'
			when risk IN ('Low - Medium') then '3.Low-Mid'
			when risk IN ('Low-Medium') then '3.Low-Mid'                                            
			when risk IN ('Medium - High') then '4.Mid-High'
			when risk IN ('Medium-High') then '4.Mid-High'                                          
			when risk IN ('High') then '5.High'
			when risk IN ('Extremely High') then '6.Extremely High'
        END as risk
FROM d 
INNER JOIN t on lower(commodity)=crop 
ORDER BY risk asc, crop asc                                              
```

`params_config:`
```json
                "params_config": [
                    {
                        "required": true,
                        "sufix": "l",
                        "key": "water_column"
                    },
                    {
                        "required": true,
                        "key": "year"
                    },
                    {
                        "required": true,
                        "key": "iso"
                    },
                    {
                        "required": true,
                        "key": "countryName"
                    },
                    {
                        "required": true,
                        "key": "crop_name"
                    }
                ],
```

In [58]:
payload = {'q': "with t as (SELECT sum(area) as area, initcap(split_part(risk, ' (', 1)) as risk, crop FROM crops_risk_stats WHERE indicator='water_stress' and irrigation='irrigated' and crop in ('maize','rice','soybean','wheat') and year = 2010 and iso = 'ESP' GROUP BY risk, crop), r as (SELECT sum(value) as value, commodity FROM combined01_prepared WHERE commodity<>'All Cereals' and commodity<>'All Pulses' and impactparameter='Food Demand' and scenario='SSP2-MIRO' and year = 2010 and iso = 'ESP' GROUP BY commodity), d as (SELECT value, commodity FROM r) SELECT crop, value, (area*100/sum(area) over(partition by crop)) area_perc, CASE when risk IN ('No Data') then '1.No Data' when risk IN ('Low') then '2.Low' when risk IN ('Low - Medium') then '3.Low-Mid' when risk IN ('Low-Medium') then '3.Low-Mid' when risk IN ('Medium - High') then '4.Mid-High' when risk IN ('Medium-High') then '4.Mid-High' when risk IN ('High') then '5.High' when risk IN ('Extremely High') then '6.Extremely High' END as risk FROM d INNER JOIN t on lower(commodity)=crop ORDER BY risk asc, crop asc"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df

<Response [200]>


,area_perc,crop,risk,value
0,0.009371,maize,1.No Data,70.163628
1,0.013343,rice,1.No Data,296.881927
2,0.005690,wheat,1.No Data,3842.070312
3,5.426092,maize,2.Low,70.163628
4,3.987565,rice,2.Low,296.881927
5,2.840498,wheat,2.Low,3842.070312
6,7.020497,maize,3.Low-Mid,70.163628
7,9.457535,rice,3.Low-Mid,296.881927
8,5.769997,wheat,3.Low-Mid,3842.070312
9,10.397320,maize,4.Mid-High,70.163628


#### Percentage of crop production area by water risk in year in country
* **Widget_id:** [962fad79-ec09-4258-b233-d4934bfbbf61](https://staging-api.globalforestwatch.org/v1/widget/962fad79-ec09-4258-b233-d4934bfbbf61) 

**Old query**

`url: https://wri-rw.carto.com/api/v2/sql?=`

`Sql:`
```sql
with ff as (select *, case when {{year}}=2010  
                           then  2010 else year  
                      end as y  
            from water_risk_indicators 
            where (year={{year}} or year = 2014) and basinid in (SELECT basinid 
                                                                 FROM wri_subcatchements_v2 
                                                                 where iso='{{iso}}')),  
     wr as (SELECT basinid, indicator, label, value 
            FROM ff 
            where type='absolute' and period='year' and y={{year}} and indicator in ('seasonal_variability', 'drought_severity', 'water_stress','groundwater_stress', 'interannual_variability') and label is not null and label !='' and label !='-32767'), 
    prod as (SELECT sum(value) as value 
             FROM combined01_prepared 
             where iso='{{iso}}' and year={{year}} and  commodity<>'All Cereals' and commodity<>'All Pulses' and impactparameter='Production' and scenario = 'SSP2-MIRO' 
             order by value desc), 
       s as (SELECT sum(area) as perc, min(basin_total_area) b_area, basinid, irrigation 
             FROM all_crops_stats group by basinid, irrigation), 
       f as (select  wr.basinid, indicator, label, perc, b_area, irrigation,sum(perc) over (partition by indicator) as sdss, value  from s inner join wr on wr.basinid=s.basinid), 
       t as (select sum(perc)*100/sdss a, label, indicator, irrigation, max(value) as value_m  from f group by label, indicator, irrigation, sdss order by indicator asc, label asc ) select initcap(split_part(label, ' (', 1)) as risk, row_number() over (partition by indicator order by value_m asc) as index, replace(initcap(indicator), '_', ' ') as indicator, irrigation, prod.value*a as perc_value, value_m, a as area_perc from t, prod  where (indicator='water_stress' and irrigation='irrigated') or (indicator='drought_severity' and irrigation='rainfed') or (indicator='seasonal_variability' and irrigation='rainfed') or (indicator='interannual_variability' and irrigation='irrigated') or (indicator='groundwater_stress' and irrigation='irrigated')
```

`sql_config`
```json
                "sql_config": [
                    {
                        "key_params": [
                            {
                                "required": true,
                                "key": "year"
                            },
                            {
                                "required": true,
                                "key": "iso"
                            }
                        ],
                        "key": "where"
                    }
                ],
```

`params_config`
```json
                
                "params_config": [
                    {
                        "key": "year",
                        "required": true
                    },
                    {
                        "required": true,
                        "key": "iso"
                    },
                    {
                        "required": true,
                        "key": "countryName"
                    }
                ],
```

**New query:**

`url: https://wri-rw.carto.com/api/v2/sql?=`

`Sql:`
```sql
with a as (SELECT sum(area) as area_perc, indicator, irrigation, sum(prod) as perc_value, risk
           FROM crops_risk_stats
           WHERE year={{year}} and iso = '{{iso}}'
           GROUP BY indicator, irrigation, risk
           ORDER BY indicator, indicator, risk),
     b as (SELECT initcap(split_part(risk, ' (', 1)) as risk, replace(initcap(indicator), '_', ' ') as indicator, area_perc, perc_value, irrigation
           FROM a
           WHERE (indicator='water_stress' and irrigation='irrigated') or (indicator='seasonal_variability' and irrigation='rainfed') or 
                 (indicator='interannual_variability' and irrigation='irrigated') or 
                 (indicator='coastal_eutrophication_potential' and irrigation='irrigated')),

     c as (SELECT indicator, area_perc, perc_value, irrigation,  
                  CASE when risk IN ('No Data') then '1.No Data'
                       when risk IN ('Low') then '2.Low'
                       when risk IN ('Low - Medium') then '3.Low-Mid'
                       when risk IN ('Low-Medium') then '3.Low-Mid'
                       when risk IN ('Medium - High') then '4.Mid-High'
                       when risk IN ('Medium-High') then '4.Mid-High'
                       when risk IN ('High') then '5. High'
                       when risk IN ('Extremely High') then '6.Extremely High'
                       when risk IN ('Arid And Low Water Use') then 'Arid and Low Water Use'
                  END as risk,
                  CASE when risk IN ('No Data') then -9999
                       when risk IN ('Low') then 0
                       when risk IN ('Low - Medium') then 1
                       when risk IN ('Low-Medium') then 1
                       when risk IN ('Medium - High') then 2
                       when risk IN ('Medium-High') then 2
                       when risk IN ('High') then 3
                       when risk IN ('Extremely High') then 4
                       when risk IN ('Arid And Low Water Use') then 5
                  END as value_m

           FROM b)

SELECT indicator, area_perc, perc_value, irrigation, risk, value_m, row_number() over (partition by indicator order by value_m asc) as index 
FROM c
```

`params_config:`
```json
                
                "params_config": [
                    {
                        "key": "year",
                        "required": true
                    },
                    {
                        "required": true,
                        "key": "iso"
                    },
                    {
                        "required": true,
                        "key": "countryName"
                    }
                ],
```

In [8]:
payload = {'q': "with a as (SELECT sum(area) as area_perc, indicator, irrigation, sum(prod) as perc_value, risk FROM crops_risk_stats WHERE year=2010  and iso = 'ESP' GROUP BY indicator, irrigation, risk ORDER BY indicator, indicator, risk), b as (SELECT initcap(split_part(risk, ' (', 1)) as risk, replace(initcap(indicator), '_', ' ') as indicator, area_perc, perc_value, irrigation FROM a WHERE (indicator='water_stress' and irrigation='irrigated') or (indicator='seasonal_variability' and irrigation='rainfed') or (indicator='interannual_variability' and irrigation='irrigated') or (indicator='coastal_eutrophication_potential' and irrigation='irrigated')), c as (SELECT indicator, area_perc, perc_value, irrigation, CASE when risk IN ('No Data') then '1.No Data' when risk IN ('Low') then '2.Low' when risk IN ('Low - Medium') then '3.Low-Mid' when risk IN ('Low-Medium') then '3.Low-Mid' when risk IN ('Medium - High') then '4.Mid-High' when risk IN ('Medium-High') then '4.Mid-High' when risk IN ('High') then '5. High' when risk IN ('Extremely High') then '6.Extremely High' when risk IN ('Arid And Low Water Use') then 'Arid and Low Water Use' END as risk, CASE when risk IN ('No Data') then -9999 when risk IN ('Low') then 0 when risk IN ('Low - Medium') then 1 when risk IN ('Low-Medium') then 1 when risk IN ('Medium - High') then 2 when risk IN ('Medium-High') then 2 when risk IN ('High') then 3 when risk IN ('Extremely High') then 4 when risk IN ('Arid And Low Water Use') then 5 END as value_m FROM b) SELECT indicator, area_perc, perc_value, irrigation, risk, value_m, row_number() over (partition by indicator order by value_m asc) as index FROM c"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df

<Response [200]>


,indicator,area_perc,perc_value,irrigation,risk,value_m,index
0,Coastal Eutrophication Potential,17873.7,373182.3,irrigated,1.No Data,-9999,1
1,Coastal Eutrophication Potential,7937.3,75428.0,irrigated,2.Low,0,2
2,Coastal Eutrophication Potential,603112.1,6649003.4,irrigated,3.Low-Mid,1,3
3,Coastal Eutrophication Potential,1451208.1,16649827.9,irrigated,4.Mid-High,2,4
4,Coastal Eutrophication Potential,581874.5,7388072.8,irrigated,5. High,3,5
5,Coastal Eutrophication Potential,13525.7,199954.6,irrigated,6.Extremely High,4,6
6,Interannual Variability,1778.4,19117.5,irrigated,1.No Data,-9999,1
7,Interannual Variability,160062.5,1436456.5,irrigated,2.Low,0,2
8,Interannual Variability,1126847.9,14317589.2,irrigated,3.Low-Mid,1,3
9,Interannual Variability,970125.5,10361161.8,irrigated,4.Mid-High,2,4


***
***
## One crop (staging)
### GLOBAL 
#### Single growing crop under high water stress conditions
* **Widget_id:** [641970f9-84da-4560-a715-ad4f70a3b8f4](https://staging-api.globalforestwatch.org/v1/widget/641970f9-84da-4560-a715-ad4f70a3b8f4)

**Old query:**
    
`url: https://wri-rw.carto.com/api/v2/sql?q=`

`Sql:`

```sql
with a as (SELECT area, basinid, cell5m, crop 
           FROM crops 
           {{where}} and irrigation='irrigated'), 
     b as (SELECT basinid, ws2028tl, ws3028tl, ws4028tl, wsbs00tl 
           FROM all_crops_stats 
           {{where}} and irrigation='irrigated'), 
  agrr as (SELECT a.basinid, a.crop, b.ws2028tl, b.ws3028tl, b.ws4028tl, b.wsbs00tl, a.area as area_a 
           FROM a,b 
           WHERE a.basinid=b.basinid), 
y_2010 as (SELECT crop as commodity, sum(area_a) as area_a 
           FROM agrr 
           WHERE wsbs00tl IN ('4.Mid-High', '5.High', '6.Extremely high') 
           GROUP BY crop), 
y_2020 as (SELECT crop as commodity, sum(area_a) as area_a 
           FROM agrr 
           WHERE ws2028tl IN ('4.Mid-High', '5.High', '6.Extremely high') 
           GROUP BY crop), 
y_2030 as (SELECT crop as commodity, sum(area_a) as area_a 
           FROM agrr 
           WHERE ws3028tl IN ('4.Mid-High', '5.High', '6.Extremely high') 
           GROUP BY crop), 
y_2040 as (SELECT crop as commodity, sum(area_a) as area_a 
           FROM agrr 
           WHERE ws4028tl IN ('4.Mid-High', '5.High', '6.Extremely high') 
           GROUP BY crop), 
result as (SELECT commodity, area_a as value, 2010 as year 
           FROM y_2010 
           UNION ALL 
           SELECT commodity, area_a as value, 2020 as year 
           FROM y_2020 
           UNION ALL 
           SELECT commodity, area_a as value, 2030 as year 
           FROM y_2030 
           UNION ALL 
           SELECT commodity, area_a as value, 2040 as year 
           FROM y_2040), 
final_res as (SELECT year, commodity, value 
              FROM result 
              WHERE year={{year}}) 
              
SELECT case 
            when year > 2010 then 'will be' 
            else 'are' 
       end as verb, 
       case 
            when year > 2010 then 'in ' || to_char(year, '9999') 
            else '' 
       end as year, 
       commodity, value 
FROM final_res
```

`sql_config:`
```json
"sql_config": [
    {
        "key_params": [
            {
                "required": true,
                "key": "crop"
            }
        ],
        "key": "where"
}]
```

`params_config:`
```json
"params_config": [
    {
        "required": true,
        "key": "year"
    },
    {
        "required": true,
        "key": "commodity"
    }
]
```


**New query:**

`url: https://wri-rw.carto.com/api/v2/sql?q=`

`Sql:`

```sql
SELECT year::text, crop as commodity, sum(area) as value, 
	   CASE 
            WHEN year > 2010 then 'will be' 
            ELSE 'are' 
       END AS verb
FROM crops_risk_stats
WHERE crop = {{crop}} and irrigation = 'irrigated' and indicator = 'water_stress' and year = {{year}} and
      initcap(split_part(risk, ' (', 1)) IN ('Medium - High', 'Medium-High', 'High', 'Extremely High', 'Extremely High')
GROUP BY crop, year
```

`sql_config:`
```json
"sql_config": [
    {
        "key_params": [
            {
                "required": true,
                "key": "crop"
            }
        ],
        "key": "where"
}]
```

`params_config:`
```json
"params_config": [
    {
        "required": true,
        "key": "crop"
    },
    {
        "required": true,
        "key": "year"
    }
]
```

In [43]:
payload = {'q': "SELECT year::text, crop as commodity, sum(area) as value, CASE WHEN year > 2010 then 'will be' ELSE 'are' END AS verb FROM crops_risk_stats WHERE crop = 'banana' and irrigation = 'irrigated' and indicator = 'water_stress' and year = 2010 and initcap(split_part(risk, ' (', 1)) IN ('Medium - High', 'Medium-High', 'High', 'Extremely High', 'Extremely High') GROUP BY crop, year"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df

<Response [200]>


,commodity,value,verb,year
0,banana,167398.7,are,2010


#### Top 10 countries with the highest crop water stress scores in year
* **Widget_id:** [413a36b5-a7b4-4bb4-9491-98b5175b4e28](https://staging-api.globalforestwatch.org/v1/widget/413a36b5-a7b4-4bb4-9491-98b5175b4e28)

**Old query:**

`url: https://wri-rw.carto.com/api/v2/sql?q=`

`Sql:`

```sql
with alpha as (SELECT iso,name_iso as region, max(value) as value, 
                      CASE when crop='millet_small' then 'millet' 
                           when crop='millet_pearl' then 'millet' 
                           when crop='lentils' then 'lentils' 
                           when crop='wheat' then 'wheat' 
                           when crop='cowpeas' then 'cowpeas' 
                           when crop='barley' then 'barley' 
                           when crop='beans' then 'beans' 
                           when crop='potato' then 'potato' 
                           when crop='sorghum' then 'sorghum' 
                           when crop='pigeonpeas' then 'pigeonpeas' 
                           when crop='sweet potato' then 'sweet potato' 
                           when crop='soybean' then 'soybean' 
                           when crop='cassava' then 'cassava' 
                           when crop='rice' then 'rice' 
                           when crop='groundnut' then 'groundnut' 
                           when crop='chickpeas' then 'chickpeas' 
                           when crop='yams' then 'yams' 
                           when crop='plantain' then 'plantain' 
                           when crop='maize' then 'maize' 
                           when crop='banana' then 'banana' 
                           when crop='cassava' then 'cassava' 
                      end as crop, 
                      CASE when year=2014 then 2010 
                           when year=2020 then 2020 
                           when year=2030 then 2030 
                           when year=2040 then 2040 
                           else 2050 
                      END as year 
               FROM water_risk_rankings_v3 
               WHERE name_iso != '<Null>' and name_iso != ' ' and indicator= 'water_stress' and value is not null 
               GROUP BY iso, region, year, crop 
               ORDER BY year asc, value desc) 
SELECT * FROM alpha 
{{where}} and year={{year}} 
ORDER BY value 
DESC LIMIT 10
```

`sql_config:`
```json
                "sql_config": [
                    {
                        "key_params": [
                            {
                                "required": true,
                                "key": "crop"
                            }
                        ],
                        "key": "where"
                    }
                ],
```

`params_config:`
```json
                "params_config": [
                    {
                        "key": "crop",
                        "required": true
                    },
                    {
                        "key": "year",
                        "required": true
                    },
                    {
                        "key": "crop_name",
                        "required": true
                    }
                ],
```

**New query:**

`url: https://wri-rw.carto.com/api/v2/sql?q=`

`Sql:`

```sql
with r as (SELECT iso, region, avg(value) as value, crop, CASE when year = 2014 then 2010 
                                         when year = 2020 then 2020 
                                         when year = 2030 then 2030 
                                         when year = 2040 then 2040 
                                         else 2050 
                                    END as year
		   FROM crops_risk_rankings
           WHERE indicator = 'water_stress'
           GROUP BY iso, region, crop, year)


SELECT * FROM r
{{where}} and year={{year}}
ORDER BY value DESC
LIMIT 10
```

`sql_config:`
```json
                "sql_config": [
                    {
                        "key_params": [
                            {
                                "required": true,
                                "key": "crop"
                            }
                        ],
                        "key": "where"
                    }
                ],
```

`params_config:`
```json
                "params_config": [
                    {
                        "key": "crop",
                        "required": true
                    },
                    {
                        "key": "year",
                        "required": true
                    },
                    {
                        "key": "crop_name",
                        "required": true
                    }
                ],
```

In [46]:
payload = {'q': "with r as (SELECT iso, region, avg(value) as value, crop, CASE when year = 2014 then 2010 when year = 2020 then 2020 when year = 2030 then 2030 when year = 2040 then 2040 else 2050 END as year FROM crops_risk_rankings WHERE indicator = 'water_stress' GROUP BY iso, region, crop, year) SELECT * FROM r WHERE crop = 'banana' and year = 2010 ORDER BY value DESC LIMIT 10"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df

<Response [200]>


,crop,iso,region,value,year
0,banana,OMN,Oman,4.637842,2010
1,banana,LBN,Lebanon,4.000000,2010
2,banana,JOR,Jordan,4.000000,2010
3,banana,BHR,Bahrain,4.000000,2010
4,banana,ISR,Israel,4.000000,2010
5,banana,ESP,Spain,3.979020,2010
6,banana,YEM,Yemen,3.689317,2010
7,banana,MAR,Morocco,3.533177,2010
8,banana,PAK,Pakistan,3.143757,2010
9,banana,CYM,Cayman Islands,3.000000,2010


#### Top 10 countries with the highest crop drought severity risk scores
* **Widget_id:** [43d3bfe3-802d-444b-a749-714e2569e2ae](https://staging-api.globalforestwatch.org/v1/widget/43d3bfe3-802d-444b-a749-714e2569e2ae)

**Old query:**

`url: https://wri-rw.carto.com/api/v2/sql?q=`

`Sql:`

```sql
with alpha as (SELECT iso,name_iso as region, max(value) as value, 
                      CASE when crop='millet_small' then 'millet' 
                           when crop='millet_pearl' then 'millet' 
                           when crop='lentils' then 'lentils' 
                           when crop='wheat' then 'wheat' 
                           when crop='cowpeas' then 'cowpeas' 
                           when crop='barley' then 'barley' 
                           when crop='beans' then 'beans' 
                           when crop='potato' then 'potato' 
                           when crop='sorghum' then 'sorghum' 
                           when crop='pigeonpeas' then 'pigeonpeas' 
                           when crop='sweet potato' then 'sweet potato' 
                           when crop='soybean' then 'soybean' 
                           when crop='cassava' then 'cassava' 
                           when crop='rice' then 'rice' 
                           when crop='groundnut' then 'groundnut' 
                           when crop='chickpeas' then 'chickpeas' 
                           when crop='yams' then 'yams' 
                           when crop='plantain' then 'plantain' 
                           when crop='maize' then 'maize' 
                           when crop='banana' then 'banana' 
                           when crop='cassava' then 'cassava' 
                      end as crop, 
                      CASE when year=2014 then 2010 
                           when year=2020 then 2020 
                           when year=2030 then 2030 
                           when year=2040 then 2040 
                           else 2050 
                      END as year 
               FROM water_risk_rankings_v3 
               WHERE name_iso != '<Null>' and name_iso != ' ' and indicator= 'water_stress' and value is not null 
               GROUP BY iso, region, year, crop 
               ORDER BY year asc, value desc) 
SELECT * FROM alpha 
{{where}} and year={{year}} 
ORDER BY value 
DESC LIMIT 10
```

`sql_config:`
```json
                "sql_config": [
                    {
                        "key_params": [
                            {
                                "required": true,
                                "key": "crop"
                            }
                        ],
                        "key": "where"
                    }
                ],
```

`params_config:`
```json
                "params_config": [
                    {
                        "key": "crop",
                        "required": true
                    },
                    {
                        "key": "year",
                        "required": true
                    },
                    {
                        "key": "crop_name",
                        "required": true
                    }
                ],
```

**New query:**

`url: https://wri-rw.carto.com/api/v2/sql?q=`

`Sql:`

```sql
with r as (SELECT iso, region, avg(value) as value, crop, year
		   FROM crops_risk_rankings
           WHERE indicator = 'drought_risk'
           GROUP BY iso, region, crop, year)


SELECT * FROM r
{{where}} and year={{year}}
ORDER BY value DESC
LIMIT 10
```

`sql_config:`
```json
                "sql_config": [
                    {
                        "key_params": [
                            {
                                "required": true,
                                "key": "crop"
                            }
                        ],
                        "key": "where"
                    }
                ],
```

`params_config:`
```json
                "params_config": [
                    {
                        "key": "crop",
                        "required": true
                    },
                    {
                        "key": "year",
                        "required": true
                    },
                    {
                        "key": "crop_name",
                        "required": true
                    }
                ],
```

In [50]:
payload = {'q': "with r as (SELECT iso, region, avg(value) as value, crop, year FROM crops_risk_rankings WHERE indicator = 'drought_risk' GROUP BY iso, region, crop, year) SELECT * FROM r WHERE crop = 'banana' and year = 2014 ORDER BY value DESC LIMIT 10"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df

<Response [200]>


,crop,iso,region,value,year
0,banana,JOR,Jordan,4.184990,2014
1,banana,PAK,Pakistan,3.986310,2014
2,banana,MAR,Morocco,3.915313,2014
3,banana,ISR,Israel,3.861804,2014
4,banana,IND,India,3.789674,2014
5,banana,BGD,Bangladesh,3.752813,2014
6,banana,IDN,Indonesia,3.674356,2014
7,banana,TGO,Togo,3.629455,2014
8,banana,LBN,Lebanon,3.624122,2014
9,banana,YEM,Yemen,3.463598,2014


#### On average risk of water stress will increase in areas where crop is grown by x through year
* **Widget_id:** [d171427d-ad95-4140-a9a4-7c24ea1019da](https://staging-api.globalforestwatch.org/v1/widget/d171427d-ad95-4140-a9a4-7c24ea1019da)

**New query:**

First we create an intermediate table called `crops_risk_change`:

```sql
with ws as (SELECT r.crop, r.irrigation, indicator, year, value
            FROM water_risk_indicators_projections s
            LEFT JOIN crops_projected r on s.basinid=r.basinid
            WHERE type = 'change_from_baseline' and indicator = 'water_stress' and scenario = 'business_as_usual' and label != 'No data' and irrigation = 'irrigated'),             
	pws as (SELECT indicator, year, round(avg((value-1.)*100)) perc, crop, irrigation
			FROM ws
			GROUP BY indicator, year, crop, irrigation),
     sv as (SELECT r.crop, r.irrigation, indicator, year, value
            FROM water_risk_indicators_projections s
            LEFT JOIN crops_projected r on s.basinid=r.basinid
            WHERE type = 'change_from_baseline' and indicator = 'seasonal_variability' and scenario = 'business_as_usual' and label != 'No data' and irrigation = 'irrigated'),             
	psv as (SELECT indicator, year, round(avg((value-1.)*100)) perc, crop, irrigation
			FROM sv
			GROUP BY indicator, year, crop, irrigation)

SELECT * FROM pws
UNION ALL
SELECT * FROM psv
```

In [55]:
payload = {'q': "SELECT * FROM crops_risk_change"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df.head()

<Response [200]>


,cartodb_id,crop,indicator,irrigation,perc,the_geom,the_geom_webmercator,year
0,1,arabica coffee,water_stress,irrigated,19,NaN,NaN,2020
1,2,banana,water_stress,irrigated,22,NaN,NaN,2020
2,3,barley,water_stress,irrigated,19,NaN,NaN,2020
3,4,bean,water_stress,irrigated,12,NaN,NaN,2020
4,5,cassava,water_stress,irrigated,14,NaN,NaN,2020


`url: https://wri-rw.carto.com/api/v2/sql?=`

`Sql:`
```sql
SELECT indicator, year, perc, crop 
FROM crops_risk_change
WHERE year = {{year}} and crop = '{{crop}}' and indicator = 'water_stress'
```

`params_config:`
```json
                "params_config": [
                    {
                        "key": "crop",
                        "required": true
                    },
                    {
                        "key": "year",
                        "required": true
                    }
                ],
```

In [56]:
payload = {'q': "SELECT indicator, year, perc, crop FROM crops_risk_change WHERE year = 2030 and crop = 'banana' and indicator = 'water_stress'"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df

<Response [200]>


,crop,indicator,perc,year
0,banana,water_stress,51,2030


#### On average risk of seasonal variability will increase in areas where crop is grown by x through year
* **Widget_id:** [b98c1774-9ce8-4ae5-b3e3-6431cc94921f](https://staging-api.globalforestwatch.org/v1/widget/b98c1774-9ce8-4ae5-b3e3-6431cc94921f)

**New query:**

`url: https://wri-rw.carto.com/api/v2/sql?=`

`Sql:`
```sql
SELECT indicator, year, perc, crop 
FROM crops_risk_change
WHERE year = {{year}} and crop = '{{crop}}' and indicator = 'seasonal_variability'
```

`params_config:`
```json
                "params_config": [
                    {
                        "key": "crop",
                        "required": true
                    },
                    {
                        "key": "year",
                        "required": true
                    }
                ],
```

In [57]:
payload = {'q': "SELECT indicator, year, perc, crop FROM crops_risk_change WHERE year = 2030 and crop = 'banana' and indicator = 'seasonal_variability'"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df

<Response [200]>


,crop,indicator,perc,year
0,banana,seasonal_variability,2,2030


#### Percentage of global production area of crop by water risk in year
* **Widget_id:** [c1d1143f-af7d-4e18-990b-b8553423e5df](https://staging-api.globalforestwatch.org/v1/widget/c1d1143f-af7d-4e18-990b-b8553423e5df)

**New query:**

`url: https://wri-rw.carto.com/api/v2/sql?=`

`Sql:`
```sql
with a as (SELECT sum(area) as area_perc, indicator, irrigation, sum(prod) as perc_value, risk
           FROM crops_risk_stats
           WHERE year={{year}} and crop='{{crop}}'
           GROUP BY indicator, irrigation, risk
           ORDER BY indicator, indicator, risk),
     b as (SELECT initcap(split_part(risk, ' (', 1)) as risk, replace(initcap(indicator), '_', ' ') as indicator, area_perc, perc_value, irrigation
           FROM a
           WHERE (indicator='water_stress' and irrigation='irrigated') or (indicator='seasonal_variability' and irrigation='rainfed') or 
                 (indicator='interannual_variability' and irrigation='irrigated') or 
                 (indicator='coastal_eutrophication_potential' and irrigation='irrigated')),

     c as (SELECT indicator, area_perc, perc_value, irrigation,  
                  CASE when risk IN ('No Data') then '1.No Data'
                       when risk IN ('Low') then '2.Low'
                       when risk IN ('Low - Medium') then '3.Low-Mid'
                       when risk IN ('Low-Medium') then '3.Low-Mid'
                       when risk IN ('Medium - High') then '4.Mid-High'
                       when risk IN ('Medium-High') then '4.Mid-High'
                       when risk IN ('High') then '5. High'
                       when risk IN ('Extremely High') then '6.Extremely High'
                       when risk IN ('Arid And Low Water Use') then 'Arid and Low Water Use'
                  END as risk,
                  CASE when risk IN ('No Data') then -9999
                       when risk IN ('Low') then 0
                       when risk IN ('Low - Medium') then 1
                       when risk IN ('Low-Medium') then 1
                       when risk IN ('Medium - High') then 2
                       when risk IN ('Medium-High') then 2
                       when risk IN ('High') then 3
                       when risk IN ('Extremely High') then 4
                       when risk IN ('Arid And Low Water Use') then 5
                  END as value_m

           FROM b)

SELECT indicator, area_perc, perc_value, irrigation, risk, value_m, row_number() over (partition by indicator order by value_m asc) as index 
FROM c
```

`sql_config:`
```json
                "sql_config": [
                    {
                        "key_params": [
                            {
                                "required": true,
                                "key": "crop"
                            }
                        ],
                        "key": "where"
                    }
                ],
```

`params_config:`
```json
                "params_config": [
                    {
                        "key": "crop",
                        "required": true
                    },
                    {
                        "key": "year",
                        "required": true
                    },
                    {
                        "key": "crop_name",
                        "required": true
                    }
                ],
```

In [9]:
payload = {'q': "with a as (SELECT sum(area) as area_perc, indicator, irrigation, sum(prod) as perc_value, risk FROM crops_risk_stats WHERE year=2010 and crop='banana' GROUP BY indicator, irrigation, risk ORDER BY indicator, indicator, risk), b as (SELECT initcap(split_part(risk, ' (', 1)) as risk, replace(initcap(indicator), '_', ' ') as indicator, area_perc, perc_value, irrigation FROM a WHERE (indicator='water_stress' and irrigation='irrigated') or (indicator='seasonal_variability' and irrigation='rainfed') or (indicator='interannual_variability' and irrigation='irrigated') or (indicator='coastal_eutrophication_potential' and irrigation='irrigated')), c as (SELECT indicator, area_perc, perc_value, irrigation, CASE when risk IN ('No Data') then '1.No Data' when risk IN ('Low') then '2.Low' when risk IN ('Low - Medium') then '3.Low-Mid' when risk IN ('Low-Medium') then '3.Low-Mid' when risk IN ('Medium - High') then '4.Mid-High' when risk IN ('Medium-High') then '4.Mid-High' when risk IN ('High') then '5. High' when risk IN ('Extremely High') then '6.Extremely High' when risk IN ('Arid And Low Water Use') then 'Arid and Low Water Use' END as risk, CASE when risk IN ('No Data') then -9999 when risk IN ('Low') then 0 when risk IN ('Low - Medium') then 1 when risk IN ('Low-Medium') then 1 when risk IN ('Medium - High') then 2 when risk IN ('Medium-High') then 2 when risk IN ('High') then 3 when risk IN ('Extremely High') then 4 when risk IN ('Arid And Low Water Use') then 5 END as value_m FROM b) SELECT indicator, area_perc, perc_value, irrigation, risk, value_m, row_number() over (partition by indicator order by value_m asc) as index  FROM c"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df

<Response [200]>


,indicator,area_perc,perc_value,irrigation,risk,value_m,index
0,Coastal Eutrophication Potential,879.4,28931.4,irrigated,1.No Data,-9999,1
1,Coastal Eutrophication Potential,287532.0,11800044.8,irrigated,2.Low,0,2
2,Coastal Eutrophication Potential,106147.4,2728930.0,irrigated,3.Low-Mid,1,3
3,Coastal Eutrophication Potential,48962.9,910076.5,irrigated,4.Mid-High,2,4
4,Coastal Eutrophication Potential,25407.8,504927.1,irrigated,5. High,3,5
5,Coastal Eutrophication Potential,25789.9,756309.6,irrigated,6.Extremely High,4,6
6,Interannual Variability,879.4,28931.4,irrigated,1.No Data,-9999,1
7,Interannual Variability,134596.1,4003694.2,irrigated,2.Low,0,2
8,Interannual Variability,154523.1,4282027.0,irrigated,3.Low-Mid,1,3
9,Interannual Variability,61116.5,1647247.2,irrigated,4.Mid-High,2,4


***
### Country
#### Country summary crop

**Old query:**

`url: https://wri-rw.carto.com/api/v2/sql?q=`

`Sql:`

```sql
with b as (SELECT * 
           FROM water_risk_rankings_v3 
           WHERE irrigation='irrigated' AND indicator='water_stress' {{and}} and value is not null 
           ORDER BY iso asc),
     c as (SELECT * 
           FROM water_risk_rankings_v3 
           WHERE irrigation='rainfed' AND indicator='drought_stress' {{and}} and value is not null 
           ORDER BY iso asc)
  
SELECT impactparameter AS name, sum(value) AS value, null as label
FROM combined01_prepared 
WHERE impactparameter in ('Area') and scenario = 'SSP2-MIRO' {{and1}} 
GROUP BY impactparameter

UNION ALL

SELECT impactparameter AS name, avg(value) AS value, null as label
FROM combined01_prepared 
WHERE impactparameter in ('Yield') and scenario = 'SSP2-MIRO' {{and1}} 
GROUP BY impactparameter

UNION ALL

SELECT impactparameter AS name, sum(value) AS value, null as label
FROM combined01_prepared 
WHERE impactparameter in ('Share Pop. at risk of hunger') and scenario = 'SSP2-MIRO' {{and2}} 
GROUP BY impactparameter

UNION ALL

SELECT 'Irrigated Area Water Stress Score' as name, value as value, label from b

UNION ALL

SELECT 'Rainfed Area Drought Severity Risk Score' as name, value as value, label from c
```

`sql_config:`
```json
"sql_config": [
  {
    "key": 'and',
    "key_params": [
      { "key": 'iso' },
      { "key": 'crop' },
      { "key": 'year', "dictionary": 'widget-2014' }
    ]
  },
  {
    "key": 'and1',
    "key_params": [
      { "key": 'iso' },
      { "key": 'irrigation' },
      { "key": 'year', "dictionary": 'food' },
      { "key": 'crop' }
    ]
  },
  {
    "key": 'and2',
    "key_params": [
      { "key": 'iso' },
      { "key": 'year', "dictionary": 'food' }
    ]
  }
];
```

**New query:**

`url: https://wri-rw.carto.com/api/v2/sql?q=`

`Sql:`
```sql
with a as (SELECT * FROM crops_risk_rankings
           WHERE irrigation='irrigated' AND indicator='water_stress' {{and}} and value is not null 
           ORDER BY iso asc),  
     b as (SELECT * 
           FROM crops_risk_rankings 
           WHERE irrigation='rainfed' AND indicator='drought_risk' {{and}} and value is not null 
           ORDER BY iso asc) 
        

SELECT 'Area' as name, sum(area) as value, null as label 
FROM crops_risk_stats
WHERE indicator='water_stress' {{and1}}           
UNION ALL 
SELECT 'Yield' as name, sum(prod)/sum(area) as value, null as label 
FROM crops_risk_stats
WHERE indicator='water_stress' {{and1}}            
UNION ALL 
SELECT impactparameter AS name, sum(value) AS value, null as label  
FROM combined01_prepared_new
WHERE impactparameter in ('Share Pop. at risk of hunger') and scenario = 'SSP2-MIRO' {{and2}} 
GROUP BY impactparameter 
UNION ALL  
SELECT 'Irrigated Area Water Stress Score' as name, value as value, label 
FROM a 
UNION ALL  
SELECT 'Rainfed Area Drought Severity Risk Score' as name, value as value, label 
FROM b
```

`sql_config:`
```json
"sql_config": [
  {
    "key": 'and',
    "key_params": [
      { "key": 'iso' },
      { "key": 'crop' },
      { "key": 'year', "dictionary": 'widget-2014' }
    ]
  },
  {
    "key": 'and1',
    "key_params": [
      { "key": 'iso' },
      { "key": 'irrigation' },
      { "key": 'year', "dictionary": 'food' },
      { "key": 'crop' }
    ]
  },
  {
    "key": 'and2',
    "key_params": [
      { "key": 'iso' },
      { "key": 'year', "dictionary": 'food' }
    ]
  }
];
```

In [64]:
payload = {'q': "with a as (SELECT * FROM crops_risk_rankings WHERE irrigation='irrigated' AND indicator='water_stress' AND iso = 'FRA' AND crop = 'maize' AND year = 2014 and value is not null ORDER BY iso asc), b as (SELECT * FROM crops_risk_rankings WHERE irrigation='rainfed' AND indicator='drought_risk' AND iso = 'FRA' AND crop = 'maize' AND year = 2014 and value is not null ORDER BY iso asc) SELECT 'Area' as name, sum(area) as value, null as label FROM crops_risk_stats WHERE indicator='water_stress' AND iso = 'FRA' AND crop = 'maize' AND year = 2010 UNION ALL SELECT 'Yield' as name, sum(prod)/sum(area) as value, null as label FROM crops_risk_stats WHERE indicator='water_stress' AND iso = 'FRA' AND crop = 'maize' AND year = 2010 UNION ALL SELECT impactparameter AS name, sum(value) AS value, null as label  FROM combined01_prepared_new WHERE impactparameter in ('Share Pop. at risk of hunger') and scenario = 'SSP2-MIRO'  AND iso = 'FRA' AND year = 2010 GROUP BY impactparameter UNION ALL SELECT 'Irrigated Area Water Stress Score' as name, value as value, label FROM a UNION ALL SELECT 'Rainfed Area Drought Severity Risk Score' as name, value as value, label FROM b"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df

<Response [200]>


,label,name,value
0,None,Area,1.606576e+06
1,None,Yield,9.308491e+00
2,None,Share Pop. at risk of hunger,5.061308e+00
3,medium-high stress,Irrigated Area Water Stress Score,2.156375e+00
4,medium-high stress,Rainfed Area Drought Severity Risk Score,3.084236e+00


#### Change over time in irrigated crop area exposed to water stress and rainfed crop area exposed to seasonal variability in country
* **Widget_id:** [69963620-4cc4-4010-9072-0315cbc71265](https://staging-api.globalforestwatch.org/v1/widget/69963620-4cc4-4010-9072-0315cbc71265)

**Old query:**

`url: https://wri-rw.carto.com/api/v2/sql?q=`

`Sql:`

```sql
with a as(SELECT basinid, crop, irrigation, iso 
          FROM crops 
          {{where}} and irrigation='irrigated'), 
    b as (SELECT basinid, ws3028tr, ws2028tr, ws4028tr, wsbs00tr 
          FROM aqueduct_projections_20150309), 
    c as (SELECT ws3028tr, ws2028tr, ws4028tr, wsbs00tr 
          FROM a,b where a.basinid=b.basinid), 
    d as (SELECT avg(ws3028tr) as ws2030, avg(ws2028tr) as ws2020, avg(ws4028tr) as ws2040, avg(wsbs00tr) as ws2010 
          FROM c), 
     e as(SELECT unnest(array[ '2010', '2020', '2030', '2040']) AS "year", 
                 unnest(array[ ws2010, ws2020, ws2030, ws2040]) AS "water_risk",
                 unnest(array[ 'Water stress', 'Water stress', 'Water stress', 'Water stress']) AS "cat"  
          FROM d 
          ORDER BY year asc), 
     f as(SELECT basinid, crop, irrigation, iso 
          FROM crops 
          {{where}} and irrigation='rainfed'), 
    g as (SELECT basinid, sv3028tr, sv2028tr, sv4028tr, svbs00tr 
          FROM aqueduct_projections_20150309), 
    h as (SELECT sv3028tr, sv2028tr, sv4028tr, svbs00tr 
          FROM f,g 
          WHERE f.basinid=g.basinid), 
    i as (SELECT avg(sv3028tr) as sv2030, avg(sv2028tr) as sv2020, avg(sv4028tr) as sv2040, avg(svbs00tr) as sv2010 
          FROM h), 
    j as (SELECT unnest(array['2010', '2020', '2030', '2040']) AS "year", 
                 unnest(array[ sv2010, sv2020, sv2030, sv2040]) AS "water_risk", 
                 unnest(array[ 'Seasonal Variability', 'Seasonal Variability', 'Seasonal Variability', 'Seasonal Variability']) AS "cat" 
          FROM i 
          ORDER BY year asc), 
  ult as (SELECT * 
          FROM e 
          WHERE  cat = 'Water stress' and water_risk is not null 
          UNION ALL 
          SELECT * 
          FROM j 
          WHERE  cat = 'Seasonal Variability' and water_risk is not null 
          ORDER BY year, cat) 
                              
SELECT year, water_risk as value, cat as impactparameter 
FROM ult

```
`params_config:`
```json
                "params_config": [
                    {
                        "required": false,
                        "key": "countryName"
                    },
                    {
                        "required": false,
                        "key": "crop_name"
                    }
                ],
```

`sql_config`
```json
                "sql_config": [
                    {
                        "key_params": [
                            {
                                "required": true,
                                "key": "iso"
                            },
                            {
                                "required": true,
                                "key": "crop"
                            }
                        ],
                        "key": "where"
                    }
                ],
```

In [66]:
payload = {'q': "with a as(SELECT basinid, crop, irrigation, iso FROM crops WHERE iso = 'FRA' AND crop = 'maize' and irrigation='irrigated'), b as (SELECT basinid, ws3028tr, ws2028tr, ws4028tr, wsbs00tr FROM aqueduct_projections_20150309), c as (select ws3028tr, ws2028tr, ws4028tr, wsbs00tr from a,b where a.basinid=b.basinid), d as (select avg(ws3028tr) as ws2030, avg(ws2028tr) as ws2020, avg(ws4028tr) as ws2040, avg(wsbs00tr) as ws2010 from c), e as(SELECT unnest(array[ '2010', '2020', '2030', '2040']) AS year, unnest(array[ ws2010, ws2020, ws2030, ws2040]) AS water_risk, unnest(array[ 'Water stress', 'Water stress', 'Water stress', 'Water stress']) AS cat  FROM d order by year asc), f as(SELECT basinid, crop, irrigation, iso FROM crops WHERE iso = 'FRA' AND crop = 'maize' and irrigation='rainfed'), g as (SELECT basinid, sv3028tr, sv2028tr, sv4028tr, svbs00tr FROM aqueduct_projections_20150309), h as (select sv3028tr, sv2028tr, sv4028tr, svbs00tr from f,g where f.basinid=g.basinid), i as (select avg(sv3028tr) as sv2030, avg(sv2028tr) as sv2020, avg(sv4028tr) as sv2040, avg(svbs00tr) as sv2010 from h), j as (SELECT unnest(array['2010', '2020', '2030', '2040']) AS year, unnest(array[ sv2010, sv2020, sv2030, sv2040]) AS water_risk, unnest(array[ 'Seasonal Variability', 'Seasonal Variability', 'Seasonal Variability', 'Seasonal Variability']) AS cat FROM i order by year asc), ult as (select * from e where  cat = 'Water stress' and water_risk is not null union all select * from j where  cat = 'Seasonal Variability' and water_risk is not null order by year, cat) select year, water_risk as value, cat as impactparameter from ult"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df

<Response [200]>


,impactparameter,value,year
0,Seasonal Variability,0.695765,2010
1,Water stress,0.178716,2010
2,Seasonal Variability,0.723032,2020
3,Water stress,0.199763,2020
4,Seasonal Variability,0.733604,2030
5,Water stress,0.229115,2030
6,Seasonal Variability,0.745224,2040
7,Water stress,0.249258,2040


**New query:**

`url: https://wri-rw.carto.com/api/v2/sql?q=`

`Sql:`

```sql
SELECT CASE when year = 2014 then 2010
            when year = 2030 then 2030
            when year = 2040 then 2040
	   END as year, value, replace(initcap(indicator), '_', ' ') as impactparameter
FROM crops_risk_rankings
{{where}} and year in (2014, 2030, 2040) and irrigation = 'irrigated' and indicator in ('water_stress', 'seasonal_variability')
ORDER BY year
```

```
`params_config:`
```json
                "params_config": [
                    {
                        "required": false,
                        "key": "countryName"
                    },
                    {
                        "required": false,
                        "key": "crop_name"
                    }
                ],
```

`sql_config`
```json
                "sql_config": [
                    {
                        "key_params": [
                            {
                                "required": true,
                                "key": "iso"
                            },
                            {
                                "required": true,
                                "key": "crop"
                            }
                        ],
                        "key": "where"
                    }
                ],
```

In [67]:
payload = {'q': "SELECT CASE when year = 2014 then 2010 when year = 2020 then 2020 when year = 2030 then 2030 when year = 2040 then 2040 END as year, value, replace(initcap(indicator), '_', ' ') as impactparameter FROM crops_risk_rankings WHERE  iso = 'ESP' and crop = 'maize' and irrigation = 'irrigated' and indicator in ('water_stress', 'seasonal_variability') ORDER BY year"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df

<Response [200]>


,impactparameter,value,year
0,Water Stress,3.485121,2010
1,Seasonal Variability,1.438981,2010
2,Water Stress,2.669405,2020
3,Seasonal Variability,1.643479,2020
4,Seasonal Variability,1.638695,2030
5,Water Stress,2.668791,2030
6,Seasonal Variability,1.656847,2040
7,Water Stress,3.105241,2040


#### Growing crops under water stress conditions
* **Widget_id:** [049f074a-3528-427d-922b-3c2320e9caf6](https://staging-api.globalforestwatch.org/v1/widget/049f074a-3528-427d-922b-3c2320e9caf6)

**New query:**

`url: https://wri-rw.carto.com/api/v2/sql?q=`

`Sql:`

```sql
SELECT year::text, crop as commodity, country, sum(area) as value, 
	   CASE 
            WHEN year > 2010 then 'will be' 
            ELSE 'are' 
       END AS verb
FROM crops_risk_stats
{{where}} and irrigation = 'irrigated' and indicator = 'water_stress' and year = {{year}} and
      initcap(split_part(risk, ' (', 1)) IN ('Medium - High', 'Medium-High', 'High', 'Extremely High', 'Extremely High')
GROUP BY crop, year, country
```

`sql_config:`
```json
"sql_config": [
    {
        "key_params": [
            {
                "required": true,
                "key": "crop"
            },
            {
                "required": true,
                "key": "iso"
            }
        ],
        "key": "where"
}]
```

`params_config:`
```json
"params_config": [
    {
        "required": true,
        "key": "year"
    }
]
```

In [71]:
payload = {'q': "SELECT year::text, crop as commodity, country, sum(area) as value, CASE WHEN year > 2010 then 'will be' ELSE 'are' END AS verb FROM crops_risk_stats WHERE iso = 'ESP' and crop = 'maize' and irrigation = 'irrigated' and indicator = 'water_stress' and year = 2010 and initcap(split_part(risk, ' (', 1)) IN ('Medium - High', 'Medium-High', 'High', 'Extremely High', 'Extremely High') GROUP BY crop, year, country"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df

<Response [200]>


,commodity,country,value,verb,year
0,maize,Spain,294274.3,are,2010


#### On average risk of water stress will increase in areas where a crop is grown through year
* **Widget_id:** [e5aef90b-79c5-4e61-a36c-7b8545864c4a](https://staging-api.globalforestwatch.org/v1/widget/e5aef90b-79c5-4e61-a36c-7b8545864c4a))

**New query:**

First we create an intermediate table called `crops_risk_change_country`:

```sql
with ws as (SELECT r.crop, r.irrigation, indicator, year, value, iso, name_cntr as country
            FROM water_risk_indicators_projections s
            LEFT JOIN crops_projected r on s.basinid=r.basinid
            WHERE type = 'change_from_baseline' and indicator = 'water_stress' and scenario = 'business_as_usual' and label != 'No data' and irrigation = 'irrigated'),             
	pws as (SELECT indicator, year, round(avg((value-1.)*100)) perc, crop, irrigation, iso, country
			FROM ws
			GROUP BY indicator, year, crop, irrigation, iso, country),
     sv as (SELECT r.crop, r.irrigation, indicator, year, value, iso, name_cntr as country
            FROM water_risk_indicators_projections s
            LEFT JOIN crops_projected r on s.basinid=r.basinid
            WHERE type = 'change_from_baseline' and indicator = 'seasonal_variability' and scenario = 'business_as_usual' and label != 'No data' and irrigation = 'irrigated'),             
	psv as (SELECT indicator, year, round(avg((value-1.)*100)) perc, crop, irrigation, iso, country
			FROM sv
			GROUP BY indicator, year, crop, irrigation, iso, country)

SELECT * FROM pws
UNION ALL
SELECT * FROM psv
```

In [72]:
payload = {'q': "SELECT * FROM crops_risk_change_country"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df.head()

<Response [200]>


,cartodb_id,country,crop,indicator,irrigation,iso,perc,the_geom,the_geom_webmercator,year
0,1,Burundi,arabica coffee,water_stress,irrigated,BDI,24,NaN,NaN,2020
1,2,Brazil,arabica coffee,water_stress,irrigated,BRA,5,NaN,NaN,2020
2,3,Colombia,arabica coffee,water_stress,irrigated,COL,31,NaN,NaN,2020
3,4,Costa Rica,arabica coffee,water_stress,irrigated,CRI,22,NaN,NaN,2020
4,5,Ecuador,arabica coffee,water_stress,irrigated,ECU,7,NaN,NaN,2020


`url: https://wri-rw.carto.com/api/v2/sql?=`

`Sql:`
```sql
SELECT indicator, year, perc, crop, iso, country 
FROM crops_risk_change_country
WHERE year = {{year}} and crop = '{{crop}}' and iso = '{{iso}}'  and indicator = 'water_stress'
```

`params_config:`
```json
                "params_config": [
                    {
                        "key": "crop",
                        "required": true
                    },
                    {
                        "key": "year",
                        "required": true
                    },
                    {
                        "key": "iso",
                        "required": true
                    }
                ],
```

In [73]:
payload = {'q': "SELECT indicator, year, perc, crop, iso, country FROM crops_risk_change_country WHERE year = 2030 and crop = 'maize' and iso = 'ESP'  and indicator = 'water_stress'"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df.head()

<Response [200]>


,country,crop,indicator,iso,perc,year
0,Spain,maize,water_stress,ESP,34,2030


#### On average risk of seasonal variability will increase in areas where a crop is grown through year
* **Widget_id:** [5e0f0039-48c2-4d17-93b6-cbb2535c22b5](https://staging-api.globalforestwatch.org/v1/widget/5e0f0039-48c2-4d17-93b6-cbb2535c22b5) 

`url: https://wri-rw.carto.com/api/v2/sql?=`

`Sql:`
```sql
SELECT indicator, year, perc, crop, iso, country 
FROM crops_risk_change_country
WHERE year = {{year}} and crop = '{{crop}}' and iso = '{{iso}}'  and indicator = 'seasonal_variability'
```

`params_config:`
```json
                "params_config": [
                    {
                        "key": "crop",
                        "required": true
                    },
                    {
                        "key": "year",
                        "required": true
                    },
                    {
                        "key": "iso",
                        "required": true
                    }
                ],
```

In [77]:
payload = {'q': "SELECT indicator, year, perc, crop, iso, country FROM crops_risk_change_country WHERE year = 2030 and crop = 'maize' and iso = 'ESP'  and indicator = 'seasonal_variability'"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df

<Response [200]>


,country,crop,indicator,iso,perc,year
0,Spain,maize,seasonal_variability,ESP,1,2030


#### Percentage of global {{crop_name}} production area by water risk in {{countryName}} in {{year}}
* **Widget_id:** [12362ca1-6f1a-43d3-8aaf-b25f1a22ed00](https://staging-api.globalforestwatch.org/v1/widget/12362ca1-6f1a-43d3-8aaf-b25f1a22ed00)

**New query:**

`url: https://wri-rw.carto.com/api/v2/sql?=`

`Sql:`
```sql
with a as (SELECT sum(area) as area_perc, indicator, irrigation, sum(prod) as perc_value, risk
           FROM crops_risk_stats
           WHERE year={{year}} and crop='{{crop}}' and iso='{{iso}}'
           GROUP BY indicator, irrigation, risk
           ORDER BY indicator, indicator, risk),
     b as (SELECT initcap(split_part(risk, ' (', 1)) as risk, replace(initcap(indicator), '_', ' ') as indicator, area_perc, perc_value, irrigation
           FROM a
           WHERE (indicator='water_stress' and irrigation='irrigated') or (indicator='seasonal_variability' and irrigation='rainfed') or 
                 (indicator='interannual_variability' and irrigation='irrigated') or 
                 (indicator='coastal_eutrophication_potential' and irrigation='irrigated')),

     c as (SELECT indicator, area_perc, perc_value, irrigation,  
                  CASE when risk IN ('No Data') then '1.No Data'
                       when risk IN ('Low') then '2.Low'
                       when risk IN ('Low - Medium') then '3.Low-Mid'
                       when risk IN ('Low-Medium') then '3.Low-Mid'
                       when risk IN ('Medium - High') then '4.Mid-High'
                       when risk IN ('Medium-High') then '4.Mid-High'
                       when risk IN ('High') then '5. High'
                       when risk IN ('Extremely High') then '6.Extremely High'
                       when risk IN ('Arid And Low Water Use') then 'Arid and Low Water Use'
                  END as risk,
                  CASE when risk IN ('No Data') then -9999
                       when risk IN ('Low') then 0
                       when risk IN ('Low - Medium') then 1
                       when risk IN ('Low-Medium') then 1
                       when risk IN ('Medium - High') then 2
                       when risk IN ('Medium-High') then 2
                       when risk IN ('High') then 3
                       when risk IN ('Extremely High') then 4
                       when risk IN ('Arid And Low Water Use') then 5
                  END as value_m

           FROM b)

SELECT indicator, area_perc, perc_value, irrigation, risk, value_m, row_number() over (partition by indicator order by value_m asc) as index 
FROM c
```

`sql_config:`
```json
                "sql_config": [
                    {
                        "key_params": [
                            {
                                "required": true,
                                "key": "crop"
                            }
                        ],
                        "key": "where"
                    }
                ],
```

`params_config:`
```json
                "params_config": [
                    {
                        "key": "year",
                        "required": true
                    },
                    {
                        "key": "crop",
                        "required": true
                    },
                    {
                        "key": "crop_name",
                        "required": true
                    },
                    {
                        "required": true,
                        "key": "iso"
                    },
                    {
                        "required": true,
                        "key": "countryName"
                    }
                ],
```

In [10]:
payload = {'q': "with a as (SELECT sum(area) as area_perc, indicator, irrigation, sum(prod) as perc_value, risk FROM crops_risk_stats WHERE year=2010 and crop='maize' and iso='ESP' GROUP BY indicator, irrigation, risk ORDER BY indicator, indicator, risk), b as (SELECT initcap(split_part(risk, ' (', 1)) as risk, replace(initcap(indicator), '_', ' ') as indicator, area_perc, perc_value, irrigation FROM a WHERE (indicator='water_stress' and irrigation='irrigated') or (indicator='seasonal_variability' and irrigation='rainfed') or (indicator='interannual_variability' and irrigation='irrigated') or (indicator='coastal_eutrophication_potential' and irrigation='irrigated')), c as (SELECT indicator, area_perc, perc_value, irrigation, CASE when risk IN ('No Data') then '1.No Data' when risk IN ('Low') then '2.Low' when risk IN ('Low - Medium') then '3.Low-Mid' when risk IN ('Low-Medium') then '3.Low-Mid' when risk IN ('Medium - High') then '4.Mid-High' when risk IN ('Medium-High') then '4.Mid-High' when risk IN ('High') then '5. High' when risk IN ('Extremely High') then '6.Extremely High' when risk IN ('Arid And Low Water Use') then 'Arid and Low Water Use' END as risk, CASE when risk IN ('No Data') then -9999 when risk IN ('Low') then 0 when risk IN ('Low - Medium') then 1 when risk IN ('Low-Medium') then 1 when risk IN ('Medium - High') then 2 when risk IN ('Medium-High') then 2 when risk IN ('High') then 3 when risk IN ('Extremely High') then 4 when risk IN ('Arid And Low Water Use') then 5 END as value_m FROM b) SELECT indicator, area_perc, perc_value, irrigation, risk, value_m, row_number() over (partition by indicator order by value_m asc) as index  FROM c"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df

<Response [200]>


,indicator,area_perc,perc_value,irrigation,risk,value_m,index
0,Coastal Eutrophication Potential,371.6,1118.2,irrigated,1.No Data,-9999,1
1,Coastal Eutrophication Potential,3596.1,13403.1,irrigated,2.Low,0,2
2,Coastal Eutrophication Potential,101157.1,1088983.4,irrigated,3.Low-Mid,1,3
3,Coastal Eutrophication Potential,209680.7,2238608.3,irrigated,4.Mid-High,2,4
4,Coastal Eutrophication Potential,21233.7,251855.5,irrigated,5. High,3,5
5,Coastal Eutrophication Potential,105.1,704.3,irrigated,6.Extremely High,4,6
6,Interannual Variability,31.5,203.7,irrigated,1.No Data,-9999,1
7,Interannual Variability,33528.4,341158.0,irrigated,2.Low,0,2
8,Interannual Variability,195526.9,1995269.7,irrigated,3.Low-Mid,1,3
9,Interannual Variability,90672.9,1051730.1,irrigated,4.Mid-High,2,4
